In [1]:
import pandas as pd
from pptx import Presentation
from pptx.dml.color import RGBColor

from pptx.util import Pt, Inches
from pptx.enum.text import PP_ALIGN
from pptx.enum.chart import XL_DATA_LABEL_POSITION
from pptx.chart.data import CategoryChartData

import numpy as np

from datetime import datetime, timedelta
import calendar

<h3>Initial config</h3>
Modify manually after putting the new excel file in the folder

In [2]:

# Variable proporcionada YYYY-MM-DD
new_date_cell = '2024-09-01'

In [3]:
pres_path = "data/2408_Auto_France_August 24_Leboncoin.pptx"

In [4]:
# Ruta al archivo Excel
# xcl = 'data/Auto FR - Data pour Export Automatisation.xlsx'
xcl = 'test/Auto FR - Data test with higher values.xlsx'

# xcl = 'test/Auto FR - Data test with 1.xlsx'
# xcl = 'test/Auto FR - Data test with 0.xlsx'

In [5]:
# Convertir new_date_cell a objeto datetime
new_date = datetime.strptime(new_date_cell, '%Y-%m-%d')

# Calcular el mes actual y el mes anterior
month = calendar.month_name[new_date.month]  # Nombre del mes actual en inglés
year = new_date.year  # Año actual

# Restar un mes para obtener el mes y año anterior
last_month_date = new_date - timedelta(days=new_date.day)  # Resta los días del mes actual
last_month = calendar.month_name[last_month_date.month]  # Nombre del mes anterior en inglés
last_year = last_month_date.year  # Año del mes anterior

# Imprimir resultados
print(f"new_date_cell: {new_date_cell}")
print(f"month: {month}")
print(f"last_month: {last_month}")
print(f"year: {year}")
print(f"last_year: {last_year}")


new_date_cell: 2024-09-01
month: September
last_month: August
year: 2024
last_year: 2024


In [6]:
import zipfile
import os
from lxml import etree
import shutil

temp_folder = "temp_pptx"

# Paso 1: Descomprimir el archivo PPTX
with zipfile.ZipFile(pres_path, 'r') as zip_ref:
    zip_ref.extractall(temp_folder)

# Paso 2: Buscar y reemplazar en TODOS los archivos XML
modified = False
for root_dir, sub_dirs, files in os.walk(temp_folder):
    for file_name in files:
        if file_name.endswith(".xml"):
            file_path = os.path.join(root_dir, file_name)
            # Leer el archivo XML
            tree = etree.parse(file_path)
            root = tree.getroot()

            # Buscar y reemplazar el texto
            for element in root.iter():
                if element.text and last_month in element.text:
                    print(f"Encontrado en {file_path}: {element.text}")
                    element.text = element.text.replace(last_month, month)
                    modified = True

            # Guardar los cambios si se modificó algo
            if modified:
                tree.write(file_path, xml_declaration=True, encoding="UTF-8", pretty_print=True)

# Paso 3: Reempaquetar el archivo PPTX modificado
temp_pptx_path = "temp_modified_pptx.pptx"
with zipfile.ZipFile(temp_pptx_path, 'w') as zip_ref:
    for folder_name, subfolders, filenames in os.walk(temp_folder):
        for filename in filenames:
            file_path = os.path.join(folder_name, filename)
            arcname = os.path.relpath(file_path, temp_folder)
            zip_ref.write(file_path, arcname)


Encontrado en temp_pptx\docProps\app.xml: Total listings per website vs LM |August 2024
Encontrado en temp_pptx\docProps\app.xml: Total listings per website vs LY | August 2024
Encontrado en temp_pptx\docProps\app.xml: Freshness ratio |August 2024
Encontrado en temp_pptx\docProps\app.xml: Shared listings between websites | August 2024
Encontrado en temp_pptx\docProps\app.xml: Private listings by website vs LM | August 2024
Encontrado en temp_pptx\docProps\app.xml: Private listings by website vs LY | August 2024
Encontrado en temp_pptx\docProps\app.xml: Dealer listings by website vs LM | August 2024
Encontrado en temp_pptx\docProps\app.xml: Dealer listings by website vs LY | August 2024
Encontrado en temp_pptx\docProps\app.xml: Dealers by website vs LM | August 2024
Encontrado en temp_pptx\docProps\app.xml: Dealers by website vs LY | August 2024
Encontrado en temp_pptx\docProps\app.xml: Dealer per website by size | August 2024
Encontrado en temp_pptx\docProps\app.xml: Shared dealers bet

<h3>Data extraction from excel and transforming it for the donut chart</h3>
change the name

In [7]:
# Nombre de la hoja 
nombre_hoja = 'Annonces particuliers'

# Leer el archivo Excel y seleccionar la hoja por su nombre
dpar = pd.read_excel(xcl, sheet_name=nombre_hoja, header=1, index_col=None).fillna(0)

if dpar.columns[0] == 'Unnamed: 0':
    dpar = dpar.drop(columns=dpar.columns[0])
dpar.columns = [pd.to_datetime(col).strftime('%B %Y') if '00:00:00' in str(col) else col for col in dpar.columns]
dpar = dpar.astype(str)

# Verificar valor obtenido
fila = dpar[dpar['Site'] == 'Total Marché dédoublonnés']

val_last_month_dd_par = int(fila.iloc[0, -1])
print(val_last_month_dd_par)

438085


In [8]:
# Nombre de la hoja 
nombre_hoja = 'Annonces pros'

# Leer el archivo Excel y seleccionar la hoja por su nombre
dpro = pd.read_excel(xcl, sheet_name=nombre_hoja, header=1, index_col=None).fillna(0)

if dpro.columns[0] == 'Unnamed: 0':
    dpro = dpro.drop(columns=dpro.columns[0])
dpro.columns = [pd.to_datetime(col).strftime('%B %Y') if '00:00:00' in str(col) else col for col in dpro.columns]
dpro = dpro.astype(str)

# Verificar valor obtenido
fila = dpro[dpro['Site'] == 'Total Marché dédoublonnés']

val_last_month_dd_pro = int(fila.iloc[0, -1]) 
print(val_last_month_dd_pro)

703188


In [9]:
total_market = val_last_month_dd_par + val_last_month_dd_pro

# Validar si total_market es 0 para evitar ZeroDivisionError
if total_market == 0:
    porcentage_par = 0
    porcentage_pro = 0
else:
    porcentage_par = round((val_last_month_dd_par / total_market) * 100)
    porcentage_pro = round((val_last_month_dd_pro / total_market) * 100)

print(porcentage_par)
print(porcentage_pro)


38
62


In [10]:
# Nombre de la hoja 
nombre_hoja = 'Annonces totales'

# Leer el archivo Excel y seleccionar la hoja por su nombre
dtm = pd.read_excel(xcl, sheet_name=nombre_hoja, header=1, index_col=None).fillna(0)

if dtm.columns[0] == 'Unnamed: 0':
    dtm = dtm.drop(columns=dtm.columns[0])
dtm.columns = [pd.to_datetime(col).strftime('%B %Y') if '00:00:00' in str(col) else col for col in dtm.columns]
dtm = dtm.astype(str)

# Verificar dedub
fila = dtm[dtm['Site'] == 'Total Marché dédoublonnés']

val_last_month_dd_tm = int(fila.iloc[0, -1])
val_before_last_month_dd_tm = int(fila.iloc[0, -2])

val_first_month_dd_tm = int(float(fila.iloc[0, 1]))

#total
fila2 = dtm[dtm['Site'] == 'Somme Panel']

val_last_month_tm = int(fila2.iloc[0, -1])
val_before_last_month_tm = int(fila2.iloc[0, -2])

print(val_last_month_dd_tm)
print(val_before_last_month_dd_tm)

print(val_last_month_tm)
print(val_before_last_month_tm)
print(val_first_month_dd_tm)

1141273
1212912
3153233
2364994
1436790


In [11]:
if val_before_last_month_dd_tm == 0:
    porcentage_ev = 0
else:
    porcentage_ev = round(((val_last_month_dd_tm - val_before_last_month_dd_tm) / val_before_last_month_dd_tm) * 100, 1)

print(porcentage_ev)


-5.9


In [12]:
def format_large_number(num):
    if num >= 1_000_000:
        return f"{num / 1_000_000:.2f} M" 
    elif num >= 1_000:
        return f"{num / 1_000:.2f} K" 
    else:
        return str(num) 

<h3>Using ppt and modifying the presentation</h3>

In [13]:
temp_aux = 'temp_modified_pptx.pptx'

In [14]:
#Presentation que se quiere editar
prs = Presentation(temp_aux)

<h3>Modifiying donut chart</h3>

In [15]:
# Load the slide and chart
diapositiva = prs.slides[3]
chart = diapositiva.shapes[0].chart

# Changing value inside the donut (esto no se toca)
cuadro_texto_donut = diapositiva.shapes[1]
cuadro_texto_donut.text_frame.clear()

# Add a textbox 1 (esto no se toca)
text_box = diapositiva.shapes.add_textbox(Inches(6), Inches(3.6), Inches(1), Inches(1))
text_frame = text_box.text_frame

# Paragraph config (esto no se toca)
paragraph = text_frame.add_paragraph()
paragraph.alignment = PP_ALIGN.CENTER
paragraph.font.bold = True
paragraph.font.size = Pt(32)
paragraph.font.color.rgb = RGBColor(0, 0, 0)
paragraph.font.name = 'Calibri'
paragraph.text = format_large_number(val_last_month_dd_tm)

# Add a textbox 2 (esto no se toca)
text_box2 = diapositiva.shapes.add_textbox(Inches(5.9), Inches(4.1), Inches(1.2), Inches(1))
text_frame2 = text_box2.text_frame

# Paragraph config (esto no se toca)
paragraph = text_frame2.add_paragraph()
paragraph.alignment = PP_ALIGN.CENTER
paragraph.font.bold = True
paragraph.font.size = Pt(18)

if porcentage_ev > 0:
    paragraph.font.color.rgb = RGBColor(0, 176, 80)
elif porcentage_ev < 0:
    paragraph.font.color.rgb = RGBColor(192, 0, 0)
else:
    paragraph.font.color.rgb = RGBColor(0, 0, 0)

paragraph.font.name = 'Calibri'
paragraph.text = f'{porcentage_ev}% vs {last_month}-{year % 100}'

# Update chart data (esto no se toca)
chart_data = CategoryChartData()
chart_data.categories = ['Pro Listings', 'Private Listings']
chart_data.add_series('Listings', (porcentage_pro, porcentage_par))
chart.replace_data(chart_data)

# Eliminar las etiquetas de datos del gráfico
plot = chart.plots[0]
for point in plot.series[0].points:
    if point.data_label is not None:
        point.data_label.text_frame.clear()  # Eliminar el texto de la etiqueta
        point.data_label.visible = False 

# Verificar y eliminar text frames existentes que contengan "Pro" o "Private"
for shape in diapositiva.shapes:
    if shape.has_text_frame:  # Verifica si la forma tiene un cuadro de texto
        text = shape.text_frame.text
        if "Pro" in text or "Private" in text:  # Busca palabras clave
            sp = shape._element  # Obtén el elemento XML subyacente
            sp.getparent().remove(sp)  # Elimina el cuadro de texto de la diapositiva


# Crear un text frame para 'Pro Listings'
pro_text_box = diapositiva.shapes.add_textbox(Inches(7.9), Inches(2.3), Inches(1.5), Inches(0.8))
pro_text_frame = pro_text_box.text_frame
pro_text_frame.clear()  # Asegurarse de que esté limpio

# Agregar el número y el signo de porcentaje
pro_paragraph = pro_text_frame.add_paragraph()
pro_paragraph.text = f'{porcentage_pro}%'  # Solo el número y el signo
pro_paragraph.alignment = PP_ALIGN.LEFT  # Alineación a la izquierda
pro_paragraph.font.bold = True
pro_paragraph.font.size = Pt(24)
pro_paragraph.font.color.rgb = RGBColor(82, 18, 98)
pro_paragraph.font.name = 'Calibri (Corps)'

# Agregar 'Pro Listings' en una nueva línea
pro_paragraph_2 = pro_text_frame.add_paragraph()
pro_paragraph_2.text = "Pro Listings"
pro_paragraph_2.alignment = PP_ALIGN.LEFT  # Alineación a la izquierda
pro_paragraph_2.font.bold = False
pro_paragraph_2.font.size = Pt(18)
pro_paragraph_2.font.color.rgb = RGBColor(82, 18, 98)
pro_paragraph_2.font.name = 'Calibri (Corps)'

# Crear un text frame para 'Private Listings'
private_text_box = diapositiva.shapes.add_textbox(Inches(3.7), Inches(5.2), Inches(1.5), Inches(0.8))
private_text_frame = private_text_box.text_frame
private_text_frame.clear()  # Asegurarse de que esté limpio

# Agregar el número y el signo de porcentaje
private_paragraph = private_text_frame.add_paragraph()
private_paragraph.text = f'{porcentage_par}%'  # Solo el número y el signo
private_paragraph.alignment = PP_ALIGN.RIGHT  # Alineación a la derecha
private_paragraph.font.bold = True
private_paragraph.font.size = Pt(24)
private_paragraph.font.color.rgb = RGBColor(82, 18, 98)
private_paragraph.font.name = 'Calibri (Corps)'

# Agregar 'Private Listings' en una nueva línea
private_paragraph_2 = private_text_frame.add_paragraph()
private_paragraph_2.text = "Private Listings"
private_paragraph_2.alignment = PP_ALIGN.RIGHT  # Alineación a la derecha
private_paragraph_2.font.bold = False
private_paragraph_2.font.size = Pt(18)
private_paragraph_2.font.color.rgb = RGBColor(82, 18, 98)
private_paragraph_2.font.name = 'Calibri (Corps)'

print("Text frames updated with left and right alignment successfully!")


Text frames updated with left and right alignment successfully!


<h3>Bar Chart data extraction from excel and tranforming</h3>

In [16]:
dtmNew = dtm.copy()  

# Aplicar la conversión a numérico a todas las columnas excepto 'Sites' (primera columna)
dtmNew.iloc[1:, 1:] = dtmNew.iloc[1:, 1:].apply(pd.to_numeric, errors='coerce').astype(int)

# Eliminar las 4 últimas filas
dtmNew = dtmNew[:-2]

# Asegurarse de que la última columna sea numérica antes de ordenar
dtmNew[dtm.columns[-1]] = pd.to_numeric(dtmNew[dtm.columns[-1]], errors='coerce')

dtmNOr = dtmNew['Site'].values[2:]
dtm_new_val_line_chart = dtmNew.iloc[:-2, -1].values

# Ordenar el DataFrame por la última columna
dtmNew = dtmNew.sort_values(by=dtm.columns[-1], ascending=False)

dtmNew.head(15)

,Site,September 2023,August 2024,September 2024
10,Somme Panel,2221622,2364994,3153233
11,Total Marché dédoublonnés,1436790,1212912,1141273
0,Argus,376197.0,383368,1064809
4,Leboncoin,754831,894554,864681
3,La Centrale,301964,317637,311519
6,ParuVendu,137850,130251,268101
9,Zoomcar,257754,189740,189816
5,OuestFrance,248046,189444,189573
2,Heycar,75675,83228,85157
1,AutoScout24,69305,76112,76925


In [17]:
val_col_1 = pd.to_numeric(dtmNew[dtmNew.columns[-1]].values[2:], errors='coerce')
name_site = dtmNew['Site'].values[2:]

val_aux_1 = pd.to_numeric(dtmNew[dtmNew.columns[-2]].values[2:], errors='coerce')
val_col_2 = (((val_col_1 - val_aux_1) / val_aux_1) * 100).round().astype(int)

val_col_3 = np.round((val_col_1 / val_last_month_dd_tm) * 100).astype(int)

val_aux_2 = ((val_col_1 / val_last_month_dd_tm) * 100).astype(float)
val_col_4 = np.round(val_aux_2 - ((val_aux_1 / val_before_last_month_dd_tm) * 100)).astype(int)

val_aux_3 =  pd.to_numeric(dtmNew[dtmNew.columns[-1]].values[:2], errors='coerce')
val_aux_4 =  pd.to_numeric(dtmNew[dtmNew.columns[-2]].values[:2], errors='coerce')

val_col_5 = (((val_aux_3 - val_aux_4) / val_aux_4) * 100).round().astype(int)

val_aux_5 = pd.to_numeric(dtmNew[dtmNew.columns[1]].values[2:], errors='coerce').astype(int)

#ignorar valores que se dividen entre 0
with np.errstate(divide='ignore', invalid='ignore'):
    val_col_6 = ((val_col_1 - val_aux_5) / val_aux_5 * 100).round().astype(float)
    val_col_6 = np.where(np.isnan(val_col_6) | np.isinf(val_col_6), 0, val_col_6).astype(int)

val_col_7 = np.round(val_aux_2 - ((val_aux_5 / val_first_month_dd_tm) * 100)).astype(int)

val_aux_6 = pd.to_numeric(dtmNew[dtmNew.columns[1]].values[:2], errors='coerce')

val_col_8 = (((val_aux_3 - val_aux_6) / val_aux_6) * 100).round().astype(int)

# Print results
print(f'val_col_6:{val_col_6}')

print(f'val_col_1:{val_col_1}')
print(f'name_site:{name_site}')

print(f'val_aux_1:{val_aux_1}')
print(f'val_col_2:{val_col_2}')

print(f'val_col_3:{val_col_3}')
print(f'val_aux_2:{val_aux_2}')
print(f'val_col_4:{val_col_4}')

print(f'val_aux_3:{val_aux_3}')
print(f'val_aux_4:{val_aux_4}')
print(f'val_col_5:{val_col_5}')

print(f'val_aux_5:{val_aux_5}')
print(f'val_col_6:{val_col_6}')

print(f'val_col_7:{val_col_7}')
print(f'val_aux_6:{val_aux_6}')
print(f'val_col_8:{val_col_8}')

val_col_6:[183  15   3  94 -26 -24  13  11   0   0]
val_col_1:[1064809  864681  311519  268101  189816  189573   85157   76925   69109
   33543]
name_site:['Argus' 'Leboncoin' 'La Centrale' 'ParuVendu' 'Zoomcar' 'OuestFrance'
 'Heycar' 'AutoScout24' 'Spoticar' 'Renault Occasions']
val_aux_1:[383368 894554 317637 130251 189740 189444  83228  76112  68043  32617]
val_col_2:[178  -3  -2 106   0   0   2   1   2   3]
val_col_3:[93 76 27 23 17 17  7  7  6  3]
val_aux_2:[93.30011312 75.76460672 27.29574782 23.49139952 16.63195397 16.61066195
  7.46158018  6.74028037  6.05543108  2.93908644]
val_col_4:[62  2  1 13  1  1  1  0  0  0]
val_aux_3:[3153233 1141273]
val_aux_4:[2364994 1212912]
val_col_5:[33 -6]
val_aux_5:[376197 754831 301964 137850 257754 248046  75675  69305      0      0]
val_col_6:[183  15   3  94 -26 -24  13  11   0   0]
val_col_7:[67 23  6 14 -1 -1  2  2  6  3]
val_aux_6:[2221622 1436790]
val_col_8:[ 42 -21]


<h3>Bar Chart data graph 1</h3>

In [18]:
#BarChart
diapositiva2 = prs.slides[5]

chart2 = diapositiva2.shapes[0].chart

# Nuevos datos para el gráfico de barras
chart_data2 = CategoryChartData()
chart_data2.categories = name_site
chart_data2.add_series(f'{month[:3]}-{year % 100}', val_col_1)

chart2.replace_data(chart_data2)

plot2 = chart2.plots[0]
plot2.has_data_labels = False

#Table 1
tabla = diapositiva2.shapes[1].table 

for i in range(len(val_col_1)):
#col 1
    formatted_value = f"{val_col_1[i]:,}".replace(",", " ")  
    
    cell_col1 = tabla.cell(i + 1, 1)  
    cell_col1.text = formatted_value  
    
    paragraph_col1 = cell_col1.text_frame.paragraphs[0]
    paragraph_col1.font.size = Pt(14) 
    paragraph_col1.font.bold = True 
    paragraph_col1.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col1.font.name = 'Calibri' 
    paragraph_col1.alignment = PP_ALIGN.CENTER

#col 2
    formatted_value2 = f"{val_col_2[i]:,}%".replace(",", " ")
    
    cell_col2 = tabla.cell(i + 1, 2)  

    if val_col_2[i] < 0:
        cell_col2.text = formatted_value2
    elif val_col_2[i] > 0:
        cell_col2.text = f'+{formatted_value2}'
    else:    
        cell_col2.text = '-'
    
    paragraph_col2 = cell_col2.text_frame.paragraphs[0]
    paragraph_col2.font.size = Pt(14) 
    paragraph_col2.font.bold = True

    if val_col_2[i] > 0:
        paragraph_col2.font.color.rgb = RGBColor(0, 176, 80) 
    elif val_col_2[i] < 0:
        paragraph_col2.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col2.font.color.rgb = RGBColor(0, 0, 0)

    paragraph_col2.font.name = 'Calibri'
    paragraph_col2.alignment = PP_ALIGN.CENTER

#col 3
    formatted_value3 = f"{val_col_3[i]:,}%".replace(",", " ")  
    
    cell_col3 = tabla.cell(i + 1, 3)  
    cell_col3.text = formatted_value3  
    
    paragraph_col3 = cell_col3.text_frame.paragraphs[0]
    paragraph_col3.font.size = Pt(14) 
    paragraph_col3.font.bold = True 
    paragraph_col3.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col3.font.name = 'Calibri' 
    paragraph_col3.alignment = PP_ALIGN.CENTER

#col 4
    formatted_value4 = f"{val_col_4[i]:,}%".replace(",", " ")  
    
    cell_col4 = tabla.cell(i + 1, 4)  

    if val_col_4[i] < 0:
        cell_col4.text = formatted_value4
    elif val_col_4[i] > 0:
        cell_col4.text = f'+{formatted_value4}'
    else:    
        cell_col4.text = '-'
    
    paragraph_col4 = cell_col4.text_frame.paragraphs[0]
    paragraph_col4.font.size = Pt(14) 
    paragraph_col4.font.bold = True 

    if val_col_4[i] > 0:
        paragraph_col4.font.color.rgb = RGBColor(0, 176, 80) 
    elif val_col_4[i] < 0:
        paragraph_col4.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col4.font.color.rgb = RGBColor(0, 0, 0)
    
    paragraph_col4.font.name = 'Calibri' 
    paragraph_col4.alignment = PP_ALIGN.CENTER


#col names 
# Extraer mes y año de la variable new_date_cell
new_date = datetime.strptime(new_date_cell, '%Y-%m-%d')
current_month = new_date.strftime('%b')  
current_year = new_date.year % 100     

# Actualizar encabezados de tabla con el mes y año de new_date_cell
tabla.cell(0, 1).text = f"Stock                  {current_month}-{current_year}"
tabla.cell(0, 3).text = f"TP                            {current_month}-{current_year}"

# Ajustar el formato del texto de los encabezados
for col in [1, 3]:
    paragraph = tabla.cell(0, col).text_frame.paragraphs[0]
    paragraph.font.size = Pt(12)
    paragraph.font.bold = True
    paragraph.font.color.rgb = RGBColor(84, 21, 98)  
    paragraph.font.name = 'Calibri'
    paragraph.alignment = PP_ALIGN.CENTER

    cell = tabla.cell(0, col)
    text_frame = cell.text_frame  
    text_frame.word_wrap = True  # Forzar el ajuste de línea
    text_frame.auto_size = None

#Table 2
tabla2 = diapositiva2.shapes[6].table 

for i in range(len(val_col_5)):

    #col
        formatted_value5 = f"{val_col_5[i]:,}%".replace(",", " ")  
        
        cell_col5 = tabla2.cell(i + 0, 1)  

        if val_col_5[i] < 0:
            cell_col5.text = formatted_value5
        elif val_col_5[i] > 0:
            cell_col5.text = f'+{formatted_value5}'
        else:    
            cell_col5.text = '-'
        
        paragraph_col5 = cell_col5.text_frame.paragraphs[0]
        paragraph_col5.font.size = Pt(14) 
        paragraph_col5.font.bold = True 

        if val_col_5[i] > 0:
            paragraph_col5.font.color.rgb = RGBColor(0, 176, 80) 
        elif val_col_5[i] < 0:
            paragraph_col5.font.color.rgb = RGBColor(192, 0, 0)
        else:
            paragraph_col5.font.color.rgb = RGBColor(0, 0, 0)

        paragraph_col5.font.name = 'Calibri' 
        paragraph_col5.alignment = PP_ALIGN.CENTER


# Ajustar el ancho
position_tabla = diapositiva2.shapes[1]
position_tabla.left = Inches(2.1)

tabla.columns[0].width = Inches(5.7)
tabla.columns[1].width = Inches(1.19)
tabla.columns[2].width = Inches(1.16)
tabla.columns[3].width = Inches(1.16)
tabla.columns[4].width = Inches(1.15)


<h3>Bar Chart data graph 2</h3>

In [19]:
#BarChart
diapositiva3 = prs.slides[6]

chart3 = diapositiva3.shapes[0].chart

# Nuevos datos para el gráfico de barras
chart_data3 = CategoryChartData()
chart_data3.categories = name_site
chart_data3.add_series(f'{month[:3]}-{year % 100}', val_col_1)

chart3.replace_data(chart_data2)

plot3 = chart3.plots[0]
plot3.has_data_labels = False

#Table 1
tabla3 = diapositiva3.shapes[1].table 

for i in range(len(val_col_1)):
#col 1
    formatted_value = f"{val_col_1[i]:,}".replace(",", " ")  
    
    cell_col1 = tabla3.cell(i + 1, 1)  
    cell_col1.text = formatted_value  
    
    paragraph_col1 = cell_col1.text_frame.paragraphs[0]
    paragraph_col1.font.size = Pt(14) 
    paragraph_col1.font.bold = True 
    paragraph_col1.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col1.font.name = 'Calibri' 
    paragraph_col1.alignment = PP_ALIGN.CENTER

#col 2
    formatted_value6 = f"{val_col_6[i]:,}%".replace(",", " ")
    
    cell_col6 = tabla3.cell(i + 1, 2)  

    if val_col_6[i] < 0:
        cell_col6.text = formatted_value6
    elif val_col_6[i] > 0:
        cell_col6.text = f'+{formatted_value6}'
    else:    
        cell_col6.text = '-'
    
    paragraph_col6 = cell_col6.text_frame.paragraphs[0]
    paragraph_col6.font.size = Pt(14) 
    paragraph_col6.font.bold = True

    if val_col_6[i] > 0:
        paragraph_col6.font.color.rgb = RGBColor(0, 176, 80) 
    elif val_col_6[i] < 0:
        paragraph_col6.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col6.font.color.rgb = RGBColor(0, 0, 0)

    paragraph_col6.font.name = 'Calibri'
    paragraph_col6.alignment = PP_ALIGN.CENTER

#col 3
    formatted_value3 = f"{val_col_3[i]:,}%".replace(",", " ")  
    
    cell_col3 = tabla3.cell(i + 1, 3)  
    cell_col3.text = formatted_value3  
    
    paragraph_col3 = cell_col3.text_frame.paragraphs[0]
    paragraph_col3.font.size = Pt(14) 
    paragraph_col3.font.bold = True 
    paragraph_col3.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col3.font.name = 'Calibri' 
    paragraph_col3.alignment = PP_ALIGN.CENTER

#col 4
    formatted_value7 = f"{val_col_7[i]:,}%".replace(",", " ")  
    
    cell_col7 = tabla3.cell(i + 1, 4)  

    if val_col_7[i] < 0:
        cell_col7.text = formatted_value7
    elif val_col_7[i] > 0:
        cell_col7.text = f'+{formatted_value7}'
    else:    
        cell_col7.text = '-'
    
    paragraph_col7 = cell_col7.text_frame.paragraphs[0]
    paragraph_col7.font.size = Pt(14) 
    paragraph_col7.font.bold = True 

    if val_col_7[i] > 0:
        paragraph_col7.font.color.rgb = RGBColor(0, 176, 80) 
    elif val_col_7[i] < 0:
        paragraph_col7.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col7.font.color.rgb = RGBColor(0, 0, 0)
    
    paragraph_col7.font.name = 'Calibri' 
    paragraph_col7.alignment = PP_ALIGN.CENTER


#col names 
# Actualizar encabezados de tabla con el mes y año de new_date_cell
tabla3.cell(0, 1).text = f"Stock              {current_month}-{current_year}"
tabla3.cell(0, 3).text = f"TP                     {current_month}-{current_year}"

# Ajustar el formato del texto de los encabezados
for col in [1, 3]:
    paragraph11 = tabla3.cell(0, col).text_frame.paragraphs[0]
    paragraph11.font.size = Pt(12)
    paragraph11.font.bold = True
    paragraph11.font.color.rgb = RGBColor(84, 21, 98)  
    paragraph11.font.name = 'Calibri Light'
    paragraph11.alignment = PP_ALIGN.CENTER

    cell11 = tabla3.cell(0, col)
    text_frame11 = cell11.text_frame  
    text_frame11.word_wrap = True  # Forzar el ajuste de línea
    text_frame11.auto_size = None
    
    
#Table 2
tabla5 = diapositiva3.shapes[6].table 

for i in range(len(val_col_5)):

    #col
        formatted_value8 = f"{val_col_8[i]:,}%".replace(",", " ")  
        
        cell_col8 = tabla5.cell(i + 0, 1)  

        if val_col_8[i] < 0:
            cell_col8.text = formatted_value8
        elif val_col_8[i] > 0:
            cell_col8.text = f'+{formatted_value8}'
        else:    
            cell_col8.text = '-'
        
        paragraph_col8 = cell_col8.text_frame.paragraphs[0]
        paragraph_col8.font.size = Pt(14) 
        paragraph_col8.font.bold = True 

        if val_col_8[i] > 0:
            paragraph_col8.font.color.rgb = RGBColor(0, 176, 80) 
        elif val_col_8[i] < 0:
            paragraph_col8.font.color.rgb = RGBColor(192, 0, 0)
        else:
            paragraph_col8.font.color.rgb = RGBColor(0, 0, 0)

        paragraph_col8.font.name = 'Calibri' 
        paragraph_col8.alignment = PP_ALIGN.CENTER



# Ajustar el ancho
position_tabla = diapositiva2.shapes[1]
position_tabla.left = Inches(2.1)

tabla.columns[0].width = Inches(5.7)
tabla.columns[1].width = Inches(1.19)
tabla.columns[2].width = Inches(1.16)
tabla.columns[3].width = Inches(1.16)
tabla.columns[4].width = Inches(1.15)



<h3>Extracting and transforming data for line chart 1<h3>

In [20]:
# Access the specific slide with the chart (adjust the index as necessary)
diapositiva7 = prs.slides[7]

# Locate the chart in the slide
chart7 = None
for shape in diapositiva7.shapes:
    if shape.has_chart:
        chart7 = shape.chart
        break

# Excel date conversion function
def excel_date_to_datetime(excel_serial_date):
    base_date = datetime(1899, 12, 30)  # Excel's base date
    return base_date + timedelta(days=excel_serial_date)

# Extract category labels (dates)
plot7 = chart7.plots[0]
category_labels = [c.label for c in plot7.categories]
formatted_dates = [excel_date_to_datetime(int(date)).strftime('%Y-%m-%d') for date in category_labels]
formatted_dates2 = [excel_date_to_datetime(int(date)).strftime('%b-%y') for date in category_labels]

# Append new date to the category labels
new_date_form = datetime.strptime(new_date_cell, '%Y-%m-%d')
new_formatted = new_date_form.strftime('%b-%y')
formatted_dates2.append(new_formatted)

# Ensure only the last 12 months are kept
formatted_dates2 = formatted_dates2[-13:]

# Extract series names and values
x = []
x1 = []

for i in range(10):  # Loop over all 10 series
    series = chart7.series[i]
    x.append(series.name)
    x1.append(list(series.values))

# Initialize a dictionary to hold the company data
company_data = {}

for i in range(10):
    series = plot7.series[i]
    company_name = series.name
    values = [int(v) if v is not None else 0 for v in series.values]
    company_data[company_name] = values

# Append the new data (12 months max)
for i, company in enumerate(company_data.keys()):
    if i < len(dtm_new_val_line_chart):
        company_data[company].append(dtm_new_val_line_chart[i])

    # Trim to the last 12 months for each series
    company_data[company] = company_data[company][-13:]

# Create a new CategoryChartData object
chart_data = CategoryChartData()

# Update categories to the last 12 months
chart_data.categories = formatted_dates2

# Add updated series data (trimmed to 12 months)
for company, values in company_data.items():
    chart_data.add_series(company, values)

# Replace the chart data with the updated values
chart7.replace_data(chart_data)

# Print to verify
print(company_data)
print(category_labels)
print(formatted_dates)
print(formatted_dates2)


{'Argus': [376197, 430270, 450340, 472460, 457452, 405072, 417435, 423084, 428523, 422436, 418088, 383368, np.int64(1064809)], 'AutoScout24': [69305, 69462, 72680, 75854, 78845, 81166, 80162, 77112, 79276, 80614, 79424, 76112, np.int64(76925)], 'Heycar': [75675, 74576, 77868, 78603, 82458, 83200, 84189, 80455, 83027, 84846, 80970, 83228, np.int64(85157)], 'La Centrale-Caradisiac': [301964, 310557, 322252, 334136, 306750, 319092, 328347, 322029, 330345, 326603, 321068, 317637, np.int64(311519)], 'Leboncoin': [754831, 806216, 838825, 855361, 830225, 848650, 876247, 909518, 931994, 927697, 914838, 894554, np.int64(864681)], 'OuestFrance': [248046, 264398, 272367, 284659, 309812, 299138, 301405, 191231, 190235, 191124, 181685, 189444, np.int64(189573)], 'ParuVendu': [137850, 138971, 147159, 148618, 149653, 204777, 207877, 197069, 125751, 128716, 127952, 130251, np.int64(268101)], 'Renault Occasions': [0, 0, 0, 0, 32530, 33116, 33558, 33924, 32844, 32757, 32165, 32617, np.int64(33543)], 'Sp

<h3>Extracting and transforming data for bar chart  vertical<h3>

In [21]:
# Nombre de la hoja
nombre_hoja = 'Fraicheur'

# Leer el archivo Excel y seleccionar la hoja por su nombre
dbcv = pd.read_excel(xcl, sheet_name=nombre_hoja, header=1, index_col=1)
dbcv = dbcv.loc[:, ~dbcv.columns.str.contains('Unnamed')]
dbcv.index = range(1, len(dbcv) + 1)

# Obtener el valor promedio de 'Total Panel' antes del filtrado
average_row = dbcv.loc[dbcv['Site'] == 'Total Panel']
if not average_row.empty:
    average_value = round(average_row['%'].values[0] * 100)
else:
    average_value = 0  # Valor por defecto si no se encuentra

# Orden actual del gráfico (en PowerPoint)
chart_order = [
    'Leboncoin', 'Heycar', 'La Centrale',
    'ParuVendu', 'OuestFrance', 'Zoomcar',
    'AutoScout24', 'Argus'
]

# Filtrar el DataFrame para incluir solo las filas con los nombres deseados
dbcv = dbcv[dbcv['Site'].isin(chart_order)]

# Reordenar el DataFrame de acuerdo con el gráfico
dbcv['Site'] = pd.Categorical(dbcv['Site'], categories=chart_order, ordered=True)
dbcv = dbcv.sort_values('Site')

# Eliminar filas con valores NaN en la columna '%'
dbcv = dbcv.dropna(subset=['%'])

# Extraer datos en el orden correcto
site_names = dbcv['Site'].tolist()
bottom_vals = round(dbcv['%'].fillna(0) * 100).astype(int).tolist()
top_values = [100 - val for val in bottom_vals]

# Acceder a la diapositiva específica con el gráfico
diapositiva9 = prs.slides[8]  # Slide 9 (index 8)

# Localizar el gráfico en la diapositiva
chart9 = None
for shape in diapositiva9.shapes:
    if shape.has_chart:
        chart9 = shape.chart
        break

if chart9 is None:
    raise ValueError("No chart found on slide 9. Verify the slide index or chart presence.")

# Crear datos del gráfico
chart_data = CategoryChartData()
chart_data.categories = site_names
chart_data.add_series('Bottom', bottom_vals)  # Bottom primero
chart_data.add_series('Top', top_values)  # Top después

# Reemplazar los datos del gráfico
chart9.replace_data(chart_data)

# Actualizar la línea de promedio y formato de texto
for shape in diapositiva9.shapes:
    if shape.has_text_frame and "Average" in shape.text:
        shape.text_frame.text = f"Average Panel\n{average_value}%"
        for para in shape.text_frame.paragraphs:
            para.font.bold = True
            para.font.size = Pt(12)
            para.alignment = PP_ALIGN.CENTER
            para.font.color.rgb = RGBColor(82, 21, 98)  # Morado

# Formato de los valores en las barras
for plot, values in zip(chart9.plots, [bottom_vals, top_values]):
    for series, value_list in zip(plot.series, [bottom_vals, top_values]):
        for i, (point, val) in enumerate(zip(series.points, value_list)):
            if point.data_label:
                point.data_label.text_frame.text = f"{val}%"
                point.data_label.position = XL_DATA_LABEL_POSITION.INSIDE_END

                # Formato de texto
                para = point.data_label.text_frame.paragraphs[0]
                para.font.size = Pt(13)
                para.font.bold = True
                para.font.color.rgb = RGBColor(82, 21, 98)  # Morado

<h3>Cross table data prep and slide mods</h3>

In [22]:
# Load Excel data into DataFrame, force float to int directly
sheet_name = 'Dedoub pros'
dfdg = pd.read_excel(xcl, sheet_name=sheet_name, header=1, index_col=1).reset_index()
dfdg = dfdg.loc[:, ~dfdg.columns.str.contains('^Unnamed')]

In [23]:
#obtain the diagonal values
diagonal_values = dfdg.iloc[0:, 1:].values.diagonal().astype(int)

# Calcular el % de anuncios compartidos
porcentajes = (dfdg['Total Shared Ads'] / diagonal_values) * 100
porcentajes_enteros = porcentajes.apply(
    lambda x: 0 if pd.isna(x) or x in [float('inf'), -float('inf')] else int(round(x))
)
dfdg['% Shared Ads'] = porcentajes_enteros.astype(str) + '%'

In [24]:
# Formatear números con separadores de miles
def format_number(value):
    if isinstance(value, (int, float)):
        return f"{value:,}".replace(",", " ")
    return value

dfdg = dfdg.applymap(format_number)

# Access the specific slide with the chart (adjust the index as necessary)
diapositiva19 = prs.slides[9]

# Locate the chart in the slide
chart_t = None
for shape in diapositiva19.shapes:
    if shape.has_table:
        chart_t = shape.table
        break

if chart_t:
    # Iterate through the table and replace its data
    row_count = len(chart_t.rows)
    col_count = len(chart_t.columns)

    # Prepare the data for replacement (excluding website names)
    data_columns = ["Argus", "AutoScout24", "Heycar", "La Centrale", "Leboncoin", "OuestFrance", "ParuVendu",
                     "Renault Occasions", "Spoticar", "Zoomcar", "Total Shared Ads", "% Shared Ads"]
    dfdg = dfdg.reset_index(drop=True)

    for i, row in dfdg.iterrows():
        if i + 1 >= row_count:  # Ensure we do not exceed the table rows
            break

        for j, col_name in enumerate(data_columns):
            if j + 1 >= col_count:  # Ensure we do not exceed the table columns
                break
            
            cell = chart_t.cell(i + 1, j + 1)
            cell.text = str(row[col_name])
            
            # Align text to the center
            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
            
            # Set font to Arial Narrow, size 14
            run = cell.text_frame.paragraphs[0].runs[0]
            run.font.name = 'Arial Narrow'
            run.font.size = Pt(14)
            
            # Apply bold formatting to diagonal cells
            if i == j:  # Diagonal condition
                run.font.bold = True

    print("Data replaced successfully!")
else:
    print("No table found in slide 9.")


Data replaced successfully!


C:\Users\r.claros\AppData\Local\Temp\ipykernel_15896\725504390.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfdg = dfdg.applymap(format_number)


<h3>Private listings by website vs LM and LY slides</h3>

In [25]:
# Nombre de la hoja
nombre_hoja = 'Annonces particuliers'

# Leer el archivo Excel y seleccionar la hoja por su nombre
dplw = pd.read_excel(xcl, sheet_name=nombre_hoja, header=1, index_col=0)

In [26]:
# transforming dates in the name of the columns to the format 'Month-Year'
dplw.columns = [pd.to_datetime(col).strftime('%B %Y') if '00:00:00' in str(col) else col for col in dplw.columns]

# dropping NaN columns
dplw = dplw.dropna(axis=1, how='all')

# dropping rows that have at least one 0 value
dplw = dplw[dplw['Site'].isin(['Leboncoin', 'La Centrale', 'ParuVendu', 'AutoScout24', 'Total Marché dédoublonnés', 'Somme Panel'])]

# ordering the DataFrame by the last column
dplw = dplw.sort_values(by=dplw.columns[-1], ascending=False)
print(dplw)

xfila = dplw[dplw['Site'] == 'Total Marché dédoublonnés']

xval_last_month_dd_tm = int(xfila.iloc[0, -1])
xval_before_last_month_dd_tm = int(xfila.iloc[0, -2])

xval_first_month_dd_tm = int(float(xfila.iloc[0, 1]))

print(f'xval_last_month_dd_tm: {xval_last_month_dd_tm}')
print(f'xval_before_last_month_dd_tm: {xval_before_last_month_dd_tm}')
print(f'xval_first_month_dd_tm: {xval_first_month_dd_tm}')

                          Site  September 2023  August 2024  September 2024
NaN                Somme Panel        356859.0       469989          457986
NaN  Total Marché dédoublonnés        347629.0       449248          438085
NaN                  Leboncoin        317263.0       418119          406022
NaN                La Centrale         26511.0        34511           33868
NaN                  ParuVendu          7229.0         8767            9489
NaN                AutoScout24          5856.0         8592            8607
xval_last_month_dd_tm: 438085
xval_before_last_month_dd_tm: 449248
xval_first_month_dd_tm: 347629


In [27]:
xval_col_1 = pd.to_numeric(dplw[dplw.columns[-1]].values[2:], errors='coerce')
xname_site = dplw['Site'].values[2:]

xval_aux_1 = pd.to_numeric(dplw[dplw.columns[-2]].values[2:], errors='coerce')
xval_col_2 = (((xval_col_1 - xval_aux_1) / xval_aux_1) * 100).round().astype(int)

xval_col_3 = np.round((xval_col_1 / xval_last_month_dd_tm) * 100).astype(int)

xval_aux_2 = ((xval_col_1 / xval_last_month_dd_tm) * 100).astype(float)
xval_col_4 = np.round(xval_aux_2 - ((xval_aux_1 / xval_before_last_month_dd_tm) * 100)).astype(int)

xval_aux_3 =  pd.to_numeric(dplw[dplw.columns[-1]].values[:2], errors='coerce')
xval_aux_4 =  pd.to_numeric(dplw[dplw.columns[-2]].values[:2], errors='coerce')

xval_col_5 = (((xval_aux_3 - xval_aux_4) / xval_aux_4) * 100).round().astype(int)

xval_aux_5 = pd.to_numeric(dplw[dplw.columns[1]].values[2:], errors='coerce').astype(int)

#ignorar valores que se dividen entre 0
with np.errstate(divide='ignore', invalid='ignore'):
    xval_col_6 = ((xval_col_1 - xval_aux_5) / xval_aux_5 * 100).round().astype(float)
    xval_col_6 = np.where(np.isnan(xval_col_6) | np.isinf(xval_col_6), 0, xval_col_6).astype(int)

xval_col_7 = np.round(xval_aux_2 - ((xval_aux_5 / xval_first_month_dd_tm) * 100)).astype(int)

xval_aux_6 = pd.to_numeric(dplw[dplw.columns[1]].values[:2], errors='coerce')

xval_col_8 = (((xval_aux_3 - xval_aux_6) / xval_aux_6) * 100).round().astype(int)

# Print results
print(f'val_col_6:{xval_col_6}')

print(f'val_col_1:{xval_col_1}')
print(f'name_site:{xname_site}')

print(f'val_aux_1:{xval_aux_1}')
print(f'val_col_2:{xval_col_2}')

print(f'val_col_3:{xval_col_3}')
print(f'val_aux_2:{xval_aux_2}')
print(f'val_col_4:{xval_col_4}')

print(f'val_aux_3:{xval_aux_3}')
print(f'val_aux_4:{xval_aux_4}')
print(f'val_col_5:{xval_col_5}')

print(f'val_aux_5:{xval_aux_5}')
print(f'val_col_6:{xval_col_6}')

print(f'val_col_7:{xval_col_7}')
print(f'val_aux_6:{xval_aux_6}')
print(f'val_col_8:{xval_col_8}')

val_col_6:[28 28 31 47]
val_col_1:[406022  33868   9489   8607]
name_site:['Leboncoin' 'La Centrale' 'ParuVendu' 'AutoScout24']
val_aux_1:[418119  34511   8767   8592]
val_col_2:[-3 -2  8  0]
val_col_3:[93  8  2  2]
val_aux_2:[92.6811007   7.7309198   2.16601801  1.96468722]
val_col_4:[0 0 0 0]
val_aux_3:[457986 438085]
val_aux_4:[469989 449248]
val_col_5:[-3 -2]
val_aux_5:[317263  26511   7229   5856]
val_col_6:[28 28 31 47]
val_col_7:[1 0 0 0]
val_aux_6:[356859. 347629.]
val_col_8:[28 26]


In [28]:
#BarChart
diapositiva12 = prs.slides[11]

chart12 = diapositiva12.shapes[0].chart

# Nuevos datos para el gráfico de barras
chart_data12 = CategoryChartData()
chart_data12.categories = xname_site
chart_data12.add_series(f'{month[:3]}-{year % 100}', xval_col_1)

chart12.replace_data(chart_data12)

plot12 = chart12.plots[0]
plot12.has_data_labels = False

#Table 1
tabla12 = diapositiva12.shapes[1].table 

for i in range(len(xval_col_1)):
#col 1
    formatted_value12 = f"{xval_col_1[i]:,}".replace(",", " ")  
    
    cell_col12 = tabla12.cell(i + 1, 1)  
    cell_col12.text = formatted_value12
    
    paragraph_col12 = cell_col12.text_frame.paragraphs[0]
    paragraph_col12.font.size = Pt(14) 
    paragraph_col12.font.bold = True 
    paragraph_col12.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col12.font.name = 'Calibri' 
    paragraph_col12.alignment = PP_ALIGN.CENTER

#col 2
    formatted_value13 = f"{xval_col_2[i]:,}%".replace(",", " ")
    
    cell_col13 = tabla12.cell(i + 1, 2)  

    if xval_col_2[i] < 0:
        cell_col13.text = formatted_value13
    elif xval_col_2[i] > 0:
        cell_col13.text = f'+{formatted_value13}'
    else:    
        cell_col13.text = '-'
    
    paragraph_col13 = cell_col13.text_frame.paragraphs[0]
    paragraph_col13.font.size = Pt(14) 
    paragraph_col13.font.bold = True

    if xval_col_2[i] > 0:
        paragraph_col13.font.color.rgb = RGBColor(0, 176, 80) 
    elif xval_col_2[i] < 0:
        paragraph_col13.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col13.font.color.rgb = RGBColor(0, 0, 0)

    paragraph_col13.font.name = 'Calibri'
    paragraph_col13.alignment = PP_ALIGN.CENTER

#col 3
    formatted_value14 = f"{xval_col_3[i]:,}%".replace(",", " ")  
    
    cell_col14 = tabla12.cell(i + 1, 3)  
    cell_col14.text = formatted_value14  
    
    paragraph_col14 = cell_col14.text_frame.paragraphs[0]
    paragraph_col14.font.size = Pt(14) 
    paragraph_col14.font.bold = True 
    paragraph_col14.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col14.font.name = 'Calibri' 
    paragraph_col14.alignment = PP_ALIGN.CENTER

#col 4
    formatted_value15 = f"{xval_col_4[i]:,}%".replace(",", " ")  
    
    cell_col15 = tabla12.cell(i + 1, 4)  

    if xval_col_4[i] < 0:
        cell_col15.text = formatted_value15
    elif xval_col_4[i] > 0:
        cell_col15.text = f'+{formatted_value15}'
    else:    
        cell_col15.text = '-'
    
    paragraph_col15 = cell_col15.text_frame.paragraphs[0]
    paragraph_col15.font.size = Pt(14) 
    paragraph_col15.font.bold = True 

    if xval_col_4[i] > 0:
        paragraph_col15.font.color.rgb = RGBColor(0, 176, 80) 
    elif xval_col_4[i] < 0:
        paragraph_col15.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col15.font.color.rgb = RGBColor(0, 0, 0)
    
    paragraph_col15.font.name = 'Calibri' 
    paragraph_col15.alignment = PP_ALIGN.CENTER

#col names 

# Actualizar encabezados de tabla con el mes y año de new_date_cell
tabla12.cell(0, 1).text = f"Stock                 {current_month}-{current_year}"
tabla12.cell(0, 3).text = f"TP                       {current_month}-{current_year}"

# Ajustar el formato del texto de los encabezados
for col in [1, 3]:
    paragraph12 = tabla12.cell(0, col).text_frame.paragraphs[0]
    paragraph12.font.size = Pt(12)
    paragraph12.font.bold = True
    paragraph12.font.color.rgb = RGBColor(84, 21, 98)  
    paragraph12.font.name = 'Calibri'
    paragraph12.alignment = PP_ALIGN.CENTER

    cell12 = tabla12.cell(0, col)
    text_frame12 = cell12.text_frame  
    text_frame12.word_wrap = True  # Forzar el ajuste de línea
    text_frame12.auto_size = None
    

#Table 2
tabla22 = diapositiva12.shapes[6].table 

for i in range(len(xval_col_5)):

    #col
        formatted_value20 = f"{xval_col_5[i]:,}%".replace(",", " ")  
        
        cell_col20 = tabla22.cell(i + 0, 1)  

        if xval_col_5[i] < 0:
            cell_col20.text = formatted_value20
        elif xval_col_5[i] > 0:
            cell_col20.text = f'+{formatted_value20}'
        else:    
            cell_col20.text = '-'
        
        paragraph_col20 = cell_col20.text_frame.paragraphs[0]
        paragraph_col20.font.size = Pt(14) 
        paragraph_col20.font.bold = True 

        if xval_col_5[i] > 0:
            paragraph_col20.font.color.rgb = RGBColor(0, 176, 80) 
        elif xval_col_5[i] < 0:
            paragraph_col20.font.color.rgb = RGBColor(192, 0, 0)
        else:
            paragraph_col20.font.color.rgb = RGBColor(0, 0, 0)

        paragraph_col20.font.name = 'Calibri' 
        paragraph_col20.alignment = PP_ALIGN.CENTER


# Ajustar el ancho de tablas
position_tabla15 = diapositiva12.shapes[1]
position_tabla15.left = Inches(1.9)
position_tabla15.top = Inches(1.3)

tabla22.columns[0].width = Inches(2.7)
tabla22.columns[1].width = Inches(1.7)

#ajustar height de bar chart
chart12MMA = diapositiva12.shapes[0]
chart12MMA.height = Inches(4.35)


In [29]:
#BarChart
diapositiva16 = prs.slides[12]

chart16 = diapositiva16.shapes[0].chart

# Nuevos datos para el gráfico de barras
chart_data16 = CategoryChartData()
chart_data16.categories = xname_site
chart_data16.add_series(f'{month[:3]}-{year % 100}', xval_col_1)

chart16.replace_data(chart_data16)

plot16 = chart16.plots[0]
plot16.has_data_labels = False

#Table 1
tabla16 = diapositiva16.shapes[1].table 

for i in range(len(xval_col_1)):
#col 1
    formatted_value16x = f"{xval_col_1[i]:,}".replace(",", " ")  
    
    cell_col16x = tabla16.cell(i + 1, 1)  
    cell_col16x.text = formatted_value16x  
    
    paragraph_col16x = cell_col16x.text_frame.paragraphs[0]
    paragraph_col16x.font.size = Pt(14) 
    paragraph_col16x.font.bold = True 
    paragraph_col16x.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col16x.font.name = 'Calibri' 
    paragraph_col16x.alignment = PP_ALIGN.CENTER

#col 2
    formatted_value17 = f"{xval_col_6[i]:,}%".replace(",", " ")
    
    cell_col17 = tabla16.cell(i + 1, 2)  

    if xval_col_6[i] < 0:
        cell_col17.text = formatted_value17
    elif xval_col_6[i] > 0:
        cell_col17.text = f'+{formatted_value17}'
    else:    
        cell_col17.text = '-'
    
    paragraph_col17 = cell_col17.text_frame.paragraphs[0]
    paragraph_col17.font.size = Pt(14) 
    paragraph_col17.font.bold = True

    if xval_col_6[i] > 0:
        paragraph_col17.font.color.rgb = RGBColor(0, 176, 80) 
    elif xval_col_6[i] < 0:
        paragraph_col17.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col17.font.color.rgb = RGBColor(0, 0, 0)

    paragraph_col17.font.name = 'Calibri'
    paragraph_col17.alignment = PP_ALIGN.CENTER

#col 3
    formatted_value18 = f"{xval_col_3[i]:,}%".replace(",", " ")  
    
    cell_col18 = tabla16.cell(i + 1, 3)  
    cell_col18.text = formatted_value18  
    
    paragraph_col18 = cell_col18.text_frame.paragraphs[0]
    paragraph_col18.font.size = Pt(14) 
    paragraph_col18.font.bold = True 
    paragraph_col18.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col18.font.name = 'Calibri' 
    paragraph_col18.alignment = PP_ALIGN.CENTER

#col 4
    formatted_value19 = f"{xval_col_7[i]:,}%".replace(",", " ")  
    
    cell_col19 = tabla16.cell(i + 1, 4)  

    if xval_col_7[i] < 0:
        cell_col19.text = formatted_value19
    elif xval_col_7[i] > 0:
        cell_col19.text = f'+{formatted_value19}'
    else:    
        cell_col19.text = '-'
    
    paragraph_col19 = cell_col19.text_frame.paragraphs[0]
    paragraph_col19.font.size = Pt(14) 
    paragraph_col19.font.bold = True 

    if xval_col_7[i] > 0:
        paragraph_col19.font.color.rgb = RGBColor(0, 176, 80) 
    elif xval_col_7[i] < 0:
        paragraph_col19.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col19.font.color.rgb = RGBColor(0, 0, 0)
    
    paragraph_col19.font.name = 'Calibri' 
    paragraph_col19.alignment = PP_ALIGN.CENTER


#col names 
# Actualizar encabezados de tabla con el mes y año de new_date_cell
tabla16.cell(0, 1).text = f"Stock              {current_month}-{current_year}"
tabla16.cell(0, 3).text = f"TP                     {current_month}-{current_year}"

# Ajustar el formato del texto de los encabezados
for col in [1, 3]:
    paragraph16 = tabla16.cell(0, col).text_frame.paragraphs[0]
    paragraph16.font.size = Pt(12)
    paragraph16.font.bold = True
    paragraph16.font.color.rgb = RGBColor(84, 21, 98)  
    paragraph16.font.name = 'Calibri Light'
    paragraph16.alignment = PP_ALIGN.CENTER

    cell16 = tabla16.cell(0, col)
    text_frame16 = cell16.text_frame  
    text_frame16.word_wrap = True  # Forzar el ajuste de línea
    text_frame16.auto_size = None
    
    
#Table 2
tabla55 = diapositiva16.shapes[6].table 

for i in range(len(xval_col_5)):

    #col
        formatted_value20 = f"{xval_col_8[i]:,}%".replace(",", " ")  
        
        cell_col20 = tabla55.cell(i + 0, 1)  

        if xval_col_8[i] < 0:
            cell_col20.text = formatted_value20
        elif xval_col_8[i] > 0:
            cell_col20.text = f'+{formatted_value20}'
        else:    
            cell_col20.text = '-'
        
        paragraph_col20 = cell_col20.text_frame.paragraphs[0]
        paragraph_col20.font.size = Pt(14) 
        paragraph_col20.font.bold = True 

        if xval_col_8[i] > 0:
            paragraph_col20.font.color.rgb = RGBColor(0, 176, 80) 
        elif xval_col_8[i] < 0:
            paragraph_col20.font.color.rgb = RGBColor(192, 0, 0)
        else:
            paragraph_col20.font.color.rgb = RGBColor(0, 0, 0)

        paragraph_col20.font.name = 'Calibri' 
        paragraph_col20.alignment = PP_ALIGN.CENTER

# Ajustar el ancho
position_tabla16 = diapositiva16.shapes[1]
position_tabla16.left = Inches(1.7)
position_tabla16.top = Inches(1.28)


tabla55.columns[0].width = Inches(2.7)
tabla55.columns[1].width = Inches(1.7)

#ajustar height de bar chart
chart12MMAx = diapositiva16.shapes[0]
chart12MMAx.height = Inches(4.35)

<h3>Dealer listings by website vs LM and LY slides</h3>

In [30]:
# Nombre de la hoja
nombre_hoja = 'Annonces pros'

# Leer el archivo Excel y seleccionar la hoja por su nombre
ddlw = pd.read_excel(xcl, sheet_name=nombre_hoja, header=1, index_col=0)

In [31]:
# transforming dates in the name of the columns to the format 'Month-Year'
ddlw.columns = [pd.to_datetime(col).strftime('%B %Y') if '00:00:00' in str(col) else col for col in ddlw.columns]

# dropping last two rows
ddlw = ddlw.iloc[:-2]

# replacing nan values with 0
ddlw = ddlw.fillna(0)

# ordering the DataFrame by the last column
ddlw = ddlw.sort_values(by=ddlw.columns[-1], ascending=False)

xyfila = ddlw[ddlw['Site'] == 'Total Marché dédoublonnés']

xyval_last_month_dd_tm = int(xyfila.iloc[0, -1])
xyval_before_last_month_dd_tm = int(xyfila.iloc[0, -2])

xyval_first_month_dd_tm = int(float(xyfila.iloc[0, 1]))

print(f'val_last_month_dd_tm: {xyval_last_month_dd_tm}')
print(f'val_before_last_month_dd_tm: {xyval_before_last_month_dd_tm}')
print(f'val_first_month_dd_tm: {xyval_first_month_dd_tm}')

val_last_month_dd_tm: 703188
val_before_last_month_dd_tm: 763664
val_first_month_dd_tm: 1089161


In [32]:
xyval_col_1 = pd.to_numeric(ddlw[ddlw.columns[-1]].values[2:], errors='coerce')
xyname_site = ddlw['Site'].values[2:]

xyval_aux_1 = pd.to_numeric(ddlw[ddlw.columns[-2]].values[2:], errors='coerce')
xyval_col_2 = (((xyval_col_1 - xyval_aux_1) / xyval_aux_1) * 100).round().astype(int)

xyval_col_3 = np.round((xyval_col_1 / xyval_last_month_dd_tm) * 100).astype(int)

xyval_aux_2 = ((xyval_col_1 / xyval_last_month_dd_tm) * 100).astype(float)
xyval_col_4 = np.round(xyval_aux_2 - ((xyval_aux_1 / xyval_before_last_month_dd_tm) * 100)).astype(int)

xyval_aux_3 =  pd.to_numeric(ddlw[ddlw.columns[-1]].values[:2], errors='coerce')
xyval_aux_4 =  pd.to_numeric(ddlw[ddlw.columns[-2]].values[:2], errors='coerce')

xyval_col_5 = (((xyval_aux_3 - xyval_aux_4) / xyval_aux_4) * 100).round().astype(int)

xyval_aux_5 = pd.to_numeric(ddlw[ddlw.columns[1]].values[2:], errors='coerce').astype(int)

#ignorar valores que se dividen entre 0
with np.errstate(divide='ignore', invalid='ignore'):
    xyval_col_6 = ((xyval_col_1 - xyval_aux_5) / xyval_aux_5 * 100).round().astype(float)
    xyval_col_6 = np.where(np.isnan(xyval_col_6) | np.isinf(xyval_col_6), 0, xyval_col_6).astype(int)

xyval_col_7 = np.round(xyval_aux_2 - ((xyval_aux_5 / xyval_first_month_dd_tm) * 100)).astype(int)

xyval_aux_6 = pd.to_numeric(ddlw[ddlw.columns[1]].values[:2], errors='coerce')

xyval_col_8 = (((xyval_aux_3 - xyval_aux_6) / xyval_aux_6) * 100).round().astype(int)

# Print results
print(f'val_col_6:{xyval_col_6}')

print(f'val_col_1:{xyval_col_1}')
print(f'name_site:{xyname_site}')

print(f'val_aux_1:{xyval_aux_1}')
print(f'val_col_2:{xyval_col_2}')

print(f'val_col_3:{xyval_col_3}')
print(f'val_aux_2:{xyval_aux_2}')
print(f'val_col_4:{xyval_col_4}')

print(f'val_aux_3:{xyval_aux_3}')
print(f'val_aux_4:{xyval_aux_4}')
print(f'val_col_5:{xyval_col_5}')

print(f'val_aux_5:{xyval_aux_5}')
print(f'val_col_6:{xyval_col_6}')

print(f'val_col_7:{xyval_col_7}')
print(f'val_aux_6:{xyval_aux_6}')
print(f'val_col_8:{xyval_col_8}')

val_col_6:[  5  -3   1 -26 -24  -8  13   0   8   0]
val_col_1:[458659 364809 277651 189816 189573 119819  85157  69109  68318  33543]
name_site:['Leboncoin' 'Argus' 'La Centrale' 'Zoomcar' 'OuestFrance' 'ParuVendu'
 'Heycar' 'Spoticar' 'AutoScout24' 'Renault Occasions']
val_aux_1:[476435 383368 283126 189740 189444 121484  83228  68043  67520  32617]
val_col_2:[-4 -5 -2  0  0 -1  2  2  1  3]
val_col_3:[65 52 39 27 27 17 12 10 10  5]
val_aux_2:[65.225658   51.87929828 39.4846044  26.9936347  26.9590778  17.03939771
 12.11013271  9.82795497  9.71546727  4.7701326 ]
val_col_4:[3 2 2 2 2 1 1 1 1 0]
val_aux_3:[1856454  703188]
val_aux_4:[1895005  763664]
val_col_5:[-2 -8]
val_aux_5:[437568 376197 275453 257754 248046 130621  75675      0  63449      0]
val_col_6:[  5  -3   1 -26 -24  -8  13   0   8   0]
val_col_7:[25 17 14  3  4  5  5 10  4  5]
val_aux_6:[1864763. 1089161.]
val_col_8:[  0 -35]


In [33]:
#BarChart
diapositiva12xy = prs.slides[13]

chart12xy = diapositiva12xy.shapes[0].chart

# Nuevos datos para el gráfico de barras
chart_data12xy = CategoryChartData()
chart_data12xy.categories = xyname_site
chart_data12xy.add_series(f'{month[:3]}-{year % 100}', xyval_col_1)

chart12xy.replace_data(chart_data12xy)

plot12xy = chart12xy.plots[0]
plot12xy.has_data_labels = False

#Table 1
tabla12xy = diapositiva12xy.shapes[1].table 

for i in range(len(xyval_col_1)):
#col 1
    formatted_value12xy = f"{xyval_col_1[i]:,}".replace(",", " ")  
    
    cell_col12xy = tabla12xy.cell(i + 1, 1)  
    cell_col12xy.text = formatted_value12xy
    
    paragraph_col12xy = cell_col12xy.text_frame.paragraphs[0]
    paragraph_col12xy.font.size = Pt(14) 
    paragraph_col12xy.font.bold = True 
    paragraph_col12xy.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col12xy.font.name = 'Calibri' 
    paragraph_col12xy.alignment = PP_ALIGN.CENTER

#col 2
    formatted_value13xy = f"{xyval_col_2[i]:,}%".replace(",", " ")
    
    cell_col13xy = tabla12xy.cell(i + 1, 2)  

    if xyval_col_2[i] < 0:
        cell_col13xy.text = formatted_value13xy
    elif xyval_col_2[i] > 0:
        cell_col13xy.text = f'+{formatted_value13xy}'
    else:    
        cell_col13xy.text = '-'
    
    paragraph_col13xy = cell_col13xy.text_frame.paragraphs[0]
    paragraph_col13xy.font.size = Pt(14) 
    paragraph_col13xy.font.bold = True

    if xyval_col_2[i] > 0:
        paragraph_col13xy.font.color.rgb = RGBColor(0, 176, 80) 
    elif xyval_col_2[i] < 0:
        paragraph_col13xy.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col13xy.font.color.rgb = RGBColor(0, 0, 0)

    paragraph_col13xy.font.name = 'Calibri'
    paragraph_col13xy.alignment = PP_ALIGN.CENTER

#col 3
    formatted_value14xy = f"{xyval_col_3[i]:,}%".replace(",", " ")  
    
    cell_col14xy = tabla12xy.cell(i + 1, 3)  
    cell_col14xy.text = formatted_value14xy  
    
    paragraph_col14xy = cell_col14xy.text_frame.paragraphs[0]
    paragraph_col14xy.font.size = Pt(14) 
    paragraph_col14xy.font.bold = True 
    paragraph_col14xy.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col14xy.font.name = 'Calibri' 
    paragraph_col14xy.alignment = PP_ALIGN.CENTER

#col 4
    formatted_value15xy = f"{xyval_col_4[i]:,}%".replace(",", " ")  
    
    cell_col15xy = tabla12xy.cell(i + 1, 4)  

    if xyval_col_4[i] < 0:
        cell_col15xy.text = formatted_value15xy
    elif xyval_col_4[i] > 0:
        cell_col15xy.text = f'+{formatted_value15xy}'
    else:    
        cell_col15xy.text = '-'
    
    paragraph_col15xy = cell_col15xy.text_frame.paragraphs[0]
    paragraph_col15xy.font.size = Pt(14) 
    paragraph_col15xy.font.bold = True 

    if xyval_col_4[i] > 0:
        paragraph_col15xy.font.color.rgb = RGBColor(0, 176, 80) 
    elif xyval_col_4[i] < 0:
        paragraph_col15xy.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col15xy.font.color.rgb = RGBColor(0, 0, 0)
    
    paragraph_col15xy.font.name = 'Calibri' 
    paragraph_col15xy.alignment = PP_ALIGN.CENTER

#col names 

# Actualizar encabezados de tabla con el mes y año de new_date_cell
tabla12xy.cell(0, 1).text = f"Stock                 {current_month}-{current_year}"
tabla12xy.cell(0, 3).text = f"TP                       {current_month}-{current_year}"

# Ajustar el formato del texto de los encabezados
for col in [1, 3]:
    paragraph12xy = tabla12xy.cell(0, col).text_frame.paragraphs[0]
    paragraph12xy.font.size = Pt(12)
    paragraph12xy.font.bold = True
    paragraph12xy.font.color.rgb = RGBColor(84, 21, 98)  
    paragraph12xy.font.name = 'Calibri'
    paragraph12xy.alignment = PP_ALIGN.CENTER

    cell12xy = tabla12xy.cell(0, col)
    text_frame12xy = cell12xy.text_frame  
    text_frame12xy.word_wrap = True  # Forzar el ajuste de línea
    text_frame12xy.auto_size = None
    

#Table 2
tabla22xy = diapositiva12xy.shapes[6].table 

for i in range(len(xyval_col_5)):

    #col
        formatted_value20xy = f"{xyval_col_5[i]:,}%".replace(",", " ")  
        
        cell_col20xy = tabla22xy.cell(i + 0, 1)  

        if xyval_col_5[i] < 0:
            cell_col20xy.text = formatted_value20xy
        elif xyval_col_5[i] > 0:
            cell_col20xy.text = f'+{formatted_value20xy}'
        else:    
            cell_col20xy.text = '-'
        
        paragraph_col20xy = cell_col20xy.text_frame.paragraphs[0]
        paragraph_col20xy.font.size = Pt(14) 
        paragraph_col20xy.font.bold = True 

        if xyval_col_5[i] > 0:
            paragraph_col20xy.font.color.rgb = RGBColor(0, 176, 80) 
        elif xyval_col_5[i] < 0:
            paragraph_col20xy.font.color.rgb = RGBColor(192, 0, 0)
        else:
            paragraph_col20xy.font.color.rgb = RGBColor(0, 0, 0)

        paragraph_col20xy.font.name = 'Calibri' 
        paragraph_col20xy.alignment = PP_ALIGN.CENTER

# Ajustar el ancho
position_tablaxy = diapositiva12xy.shapes[1]
position_tablaxy.left = Inches(2.1)

tabla12xy.columns[0].width = Inches(5.7)
tabla12xy.columns[1].width = Inches(1.19)
tabla12xy.columns[2].width = Inches(1.16)
tabla12xy.columns[3].width = Inches(1.16)
tabla12xy.columns[4].width = Inches(1.15)

In [34]:
#BarChart
diapositiva16xy = prs.slides[14]

chart16xy = diapositiva16xy.shapes[0].chart

# Nuevos datos para el gráfico de barras
chart_data16xy = CategoryChartData()
chart_data16xy.categories = xyname_site
chart_data16xy.add_series(f'{month[:3]}-{year % 100}', xyval_col_1)

chart16xy.replace_data(chart_data16xy)

plot16xy = chart16xy.plots[0]
plot16xy.has_data_labels = False

#Table 1
tabla16xy = diapositiva16xy.shapes[6].table 

for i in range(len(xyval_col_1)):
#col 1
    formatted_value16xy = f"{xyval_col_1[i]:,}".replace(",", " ")  
    
    cell_col16xy = tabla16xy.cell(i + 1, 1)  
    cell_col16xy.text = formatted_value16xy  
    
    paragraph_col16xy = cell_col16xy.text_frame.paragraphs[0]
    paragraph_col16xy.font.size = Pt(14) 
    paragraph_col16xy.font.bold = True 
    paragraph_col16xy.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col16xy.font.name = 'Calibri' 
    paragraph_col16xy.alignment = PP_ALIGN.CENTER

#col 2
    formatted_value17xy = f"{xyval_col_6[i]:,}%".replace(",", " ")
    
    cell_col17xy = tabla16xy.cell(i + 1, 2)  

    if xyval_col_6[i] < 0:
        cell_col17xy.text = formatted_value17xy
    elif xyval_col_6[i] > 0:
        cell_col17xy.text = f'+{formatted_value17xy}'
    else:    
        cell_col17xy.text = '-'
    
    paragraph_col17xy = cell_col17xy.text_frame.paragraphs[0]
    paragraph_col17xy.font.size = Pt(14) 
    paragraph_col17xy.font.bold = True

    if xyval_col_6[i] > 0:
        paragraph_col17xy.font.color.rgb = RGBColor(0, 176, 80) 
    elif xyval_col_6[i] < 0:
        paragraph_col17xy.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col17xy.font.color.rgb = RGBColor(0, 0, 0)

    paragraph_col17xy.font.name = 'Calibri'
    paragraph_col17xy.alignment = PP_ALIGN.CENTER

#col 3
    formatted_value18xy = f"{xyval_col_3[i]:,}%".replace(",", " ")  
    
    cell_col18xy = tabla16xy.cell(i + 1, 3)  
    cell_col18xy.text = formatted_value18xy  
    
    paragraph_col18xy = cell_col18xy.text_frame.paragraphs[0]
    paragraph_col18xy.font.size = Pt(14) 
    paragraph_col18xy.font.bold = True 
    paragraph_col18xy.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col18xy.font.name = 'Calibri' 
    paragraph_col18xy.alignment = PP_ALIGN.CENTER

#col 4
    formatted_value19xy = f"{xyval_col_7[i]:,}%".replace(",", " ")  
    
    cell_col19xy = tabla16xy.cell(i + 1, 4)  

    if xyval_col_7[i] < 0:
        cell_col19xy.text = formatted_value19xy
    elif xyval_col_7[i] > 0:
        cell_col19xy.text = f'+{formatted_value19xy}'
    else:    
        cell_col19xy.text = '-'
    
    paragraph_col19xy = cell_col19xy.text_frame.paragraphs[0]
    paragraph_col19xy.font.size = Pt(14) 
    paragraph_col19xy.font.bold = True 

    if xyval_col_7[i] > 0:
        paragraph_col19xy.font.color.rgb = RGBColor(0, 176, 80) 
    elif xyval_col_7[i] < 0:
        paragraph_col19xy.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col19xy.font.color.rgb = RGBColor(0, 0, 0)
    
    paragraph_col19xy.font.name = 'Calibri' 
    paragraph_col19xy.alignment = PP_ALIGN.CENTER

#col names 
# Actualizar encabezados de tabla con el mes y año de new_date_cell
tabla16xy.cell(0, 1).text = f"Stock              {current_month}-{current_year}"
tabla16xy.cell(0, 3).text = f"TP                     {current_month}-{current_year}"

# Ajustar el formato del texto de los encabezados
for col in [1, 3]:
    paragraph16xy = tabla16xy.cell(0, col).text_frame.paragraphs[0]
    paragraph16xy.font.size = Pt(12)
    paragraph16xy.font.bold = True
    paragraph16xy.font.color.rgb = RGBColor(84, 21, 98)  
    paragraph16xy.font.name = 'Calibri Light'
    paragraph16xy.alignment = PP_ALIGN.CENTER

    cell16xy = tabla16xy.cell(0, col)
    text_frame16 = cell16xy.text_frame  
    text_frame16.word_wrap = True
    text_frame16.auto_size = None
    
#Table 2
tabla55xy = diapositiva16xy.shapes[5].table 

for i in range(len(xyval_col_5)):

    #col
        formatted_value20xy = f"{xyval_col_8[i]:,}%".replace(",", " ")  
        
        cell_col20xy = tabla55xy.cell(i + 0, 1)  

        if xyval_col_8[i] < 0:
            cell_col20xy.text = formatted_value20xy
        elif xyval_col_8[i] > 0:
            cell_col20xy.text = f'+{formatted_value20xy}'
        else:    
            cell_col20xy.text = '-'
        
        paragraph_col20xy = cell_col20xy.text_frame.paragraphs[0]
        paragraph_col20xy.font.size = Pt(14) 
        paragraph_col20xy.font.bold = True 

        if xyval_col_8[i] > 0:
            paragraph_col20xy.font.color.rgb = RGBColor(0, 176, 80) 
        elif xyval_col_8[i] < 0:
            paragraph_col20xy.font.color.rgb = RGBColor(192, 0, 0)
        else:
            paragraph_col20xy.font.color.rgb = RGBColor(0, 0, 0)

        paragraph_col20xy.font.name = 'Calibri' 
        paragraph_col20xy.alignment = PP_ALIGN.CENTER

# Ajustar el ancho
position_tablaxyz = diapositiva16xy.shapes[6]
position_tablaxyz.left = Inches(2.1)

tabla16xy.columns[0].width = Inches(5.7)
tabla16xy.columns[1].width = Inches(1.19)
tabla16xy.columns[2].width = Inches(1.16)
tabla16xy.columns[3].width = Inches(1.16)
tabla16xy.columns[4].width = Inches(1.15)

<h3>Dealers by website vs LM and LY slides</h3>

In [35]:
# Nombre de la hoja
nombre_hoja = 'Annonceurs pro'

# Leer el archivo Excel y seleccionar la hoja por su nombre
ddw = pd.read_excel(xcl, sheet_name=nombre_hoja, header=1, index_col=0)

In [36]:
# transforming dates in the name of the columns to the format 'Month-Year'
ddw.columns = [pd.to_datetime(col).strftime('%B %Y') if '00:00:00' in str(col) else col for col in ddw.columns]

# dropping last two rows
ddw = ddw.iloc[:-2]

#dropping row that have the value 'Armisauto' and 'Autohero' in the column 'Site'
ddw = ddw[ddw['Site'] != 'Aramisauto']
ddw = ddw[ddw['Site'] != 'Autohero']

# replacing nan values with 0
ddw = ddw.fillna(0)

ddwNew = ddw.copy()  

# ordering the DataFrame by the last column
ddw = ddw.sort_values(by=ddw.columns[-1], ascending=False)


xyzfila = ddw[ddw['Site'] == 'Total Marché dédoublonnés']

xyzval_last_month_dd_tm = int(xyzfila.iloc[0, -1])
xyzval_before_last_month_dd_tm = int(xyzfila.iloc[0, -2])

xyzval_first_month_dd_tm = int(float(xyzfila.iloc[0, 1]))

print(f'val_last_month_dd_tm: {xyzval_last_month_dd_tm}')
print(f'val_before_last_month_dd_tm: {xyzval_before_last_month_dd_tm}')
print(f'val_first_month_dd_tm: {xyzval_first_month_dd_tm}')

val_last_month_dd_tm: 20215
val_before_last_month_dd_tm: 20944
val_first_month_dd_tm: 19984


In [37]:
xyzval_col_1 = pd.to_numeric(ddw[ddw.columns[-1]].values[2:], errors='coerce').astype(int)
xyzname_site = ddw['Site'].values[2:]

xyzval_aux_1 = pd.to_numeric(ddw[ddw.columns[-2]].values[2:], errors='coerce')
xyzval_col_2 = (((xyzval_col_1 - xyzval_aux_1) / xyzval_aux_1) * 100).round().astype(int)

xyzval_col_3 = np.round((xyzval_col_1 / xyzval_last_month_dd_tm) * 100).astype(int)

xyzval_aux_2 = ((xyzval_col_1 / xyzval_last_month_dd_tm) * 100).astype(float)
xyzval_col_4 = np.round(xyzval_aux_2 - ((xyzval_aux_1 / xyzval_before_last_month_dd_tm) * 100)).astype(int)

xyzval_aux_3 =  pd.to_numeric(ddw[ddw.columns[-1]].values[:2], errors='coerce')
xyzval_aux_4 =  pd.to_numeric(ddw[ddw.columns[-2]].values[:2], errors='coerce')

xyzval_col_5 = (((xyzval_aux_3 - xyzval_aux_4) / xyzval_aux_4) * 100).round().astype(int)

xyzval_aux_5 = pd.to_numeric(ddw[ddw.columns[1]].values[2:], errors='coerce').astype(int)

#ignorar valores que se dividen entre 0
with np.errstate(divide='ignore', invalid='ignore'):
    xyzval_col_6 = ((xyzval_col_1 - xyzval_aux_5) / xyzval_aux_5 * 100).round().astype(float)
    xyzval_col_6 = np.where(np.isnan(xyzval_col_6) | np.isinf(xyzval_col_6), 0, xyzval_col_6).astype(int)

xyzval_col_7 = np.round(xyzval_aux_2 - ((xyzval_aux_5 / xyzval_first_month_dd_tm) * 100)).astype(int)

xyzval_aux_6 = pd.to_numeric(ddw[ddw.columns[1]].values[:2], errors='coerce')

xyzval_col_8 = (((xyzval_aux_3 - xyzval_aux_6) / xyzval_aux_6) * 100).round().astype(int)

# Print results
print(f'val_col_6:{xyzval_col_6}')

print(f'val_col_1:{xyzval_col_1}')
print(f'name_site:{xyzname_site}')

print(f'val_aux_1:{xyzval_aux_1}')
print(f'val_col_2:{xyzval_col_2}')

print(f'val_col_3:{xyzval_col_3}')
print(f'val_aux_2:{xyzval_aux_2}')
print(f'val_col_4:{xyzval_col_4}')

print(f'val_aux_3:{xyzval_aux_3}')
print(f'val_aux_4:{xyzval_aux_4}')
print(f'val_col_5:{xyzval_col_5}')

print(f'val_aux_5:{xyzval_aux_5}')
print(f'val_col_6:{xyzval_col_6}')

print(f'val_col_7:{xyzval_col_7}')
print(f'val_aux_6:{xyzval_aux_6}')
print(f'val_col_8:{xyzval_col_8}')

val_col_6:[ 3  5  4 13 18 15  2 64  0  0]
val_col_1:[15241 14489  7502  2953  2939  1908  1490  1304  1064   414]
name_site:['Leboncoin**' 'Argus' 'La Centrale' 'Zoomcar' 'OuestFrance' 'ParuVendu'
 'AutoScout24' 'Heycar' 'Spoticar' 'Renault Occasions']
val_aux_1:[15531. 14611.  7524.  2932.  2943.  1905.  1494.  1298.  1051.   418.]
val_col_2:[-2 -1  0  1  0  0  0  0  1 -1]
val_col_3:[75 72 37 15 15  9  7  6  5  2]
val_aux_2:[75.39450903 71.67449913 37.11105615 14.60796438 14.53870888  9.43853574
  7.37076428  6.45065545  5.26341825  2.04798417]
val_col_4:[1 2 1 1 0 0 0 0 0 0]
val_aux_3:[49304. 20215.]
val_aux_4:[49707. 20944.]
val_col_5:[-1 -3]
val_aux_5:[14767 13806  7194  2603  2500  1662  1454   797     0     0]
val_col_6:[ 3  5  4 13 18 15  2 64  0  0]
val_col_7:[2 3 1 2 2 1 0 2 5 2]
val_aux_6:[44783. 19984.]
val_col_8:[10  1]


In [38]:
#BarChart
diapositiva12xyz = prs.slides[16]

chart12xyz = diapositiva12xyz.shapes[0].chart

# Nuevos datos para el gráfico de barras
chart_data12xyz = CategoryChartData()
chart_data12xyz.categories = xyzname_site
chart_data12xyz.add_series(f'{month[:3]}-{year % 100}', xyzval_col_1)

chart12xyz.replace_data(chart_data12xyz)

plot12xyz = chart12xyz.plots[0]
plot12xyz.has_data_labels = False

#Table 1
tabla12xyz = diapositiva12xyz.shapes[1].table 

for i in range(len(xyzval_col_1)):
#col 1
    formatted_value12xyz = f"{xyzval_col_1[i]:,}".replace(",", " ")  
    
    cell_col12xyz = tabla12xyz.cell(i + 1, 1)  
    cell_col12xyz.text = formatted_value12xyz
    
    paragraph_col12xyz = cell_col12xyz.text_frame.paragraphs[0]
    paragraph_col12xyz.font.size = Pt(14) 
    paragraph_col12xyz.font.bold = True 
    paragraph_col12xyz.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col12xyz.font.name = 'Calibri' 
    paragraph_col12xyz.alignment = PP_ALIGN.CENTER

#col 2
    formatted_value13xyz = f"{xyzval_col_2[i]:,}%".replace(",", " ")
    
    cell_col13xyz = tabla12xyz.cell(i + 1, 2)  

    if xyzval_col_2[i] < 0:
        cell_col13xyz.text = formatted_value13xyz
    elif xyzval_col_2[i] > 0:
        cell_col13xyz.text = f'+{formatted_value13xyz}'
    else:    
        cell_col13xyz.text = '-'
    
    paragraph_col13xyz = cell_col13xyz.text_frame.paragraphs[0]
    paragraph_col13xyz.font.size = Pt(14) 
    paragraph_col13xyz.font.bold = True

    if xyzval_col_2[i] > 0:
        paragraph_col13xyz.font.color.rgb = RGBColor(0, 176, 80) 
    elif xyzval_col_2[i] < 0:
        paragraph_col13xyz.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col13xyz.font.color.rgb = RGBColor(0, 0, 0)

    paragraph_col13xyz.font.name = 'Calibri'
    paragraph_col13xyz.alignment = PP_ALIGN.CENTER

#col 3
    formatted_value14xyz = f"{xyzval_col_3[i]:,}%".replace(",", " ")  
    
    cell_col14xyz = tabla12xyz.cell(i + 1, 3)  
    cell_col14xyz.text = formatted_value14xyz  
    
    paragraph_col14xyz = cell_col14xyz.text_frame.paragraphs[0]
    paragraph_col14xyz.font.size = Pt(14) 
    paragraph_col14xyz.font.bold = True 
    paragraph_col14xyz.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col14xyz.font.name = 'Calibri' 
    paragraph_col14xyz.alignment = PP_ALIGN.CENTER

#col 4
    formatted_value15xyz = f"{xyzval_col_4[i]:,}%".replace(",", " ")  
    
    cell_col15xyz = tabla12xyz.cell(i + 1, 4)  

    if xyzval_col_4[i] < 0:
        cell_col15xyz.text = formatted_value15xyz
    elif xyzval_col_4[i] > 0:
        cell_col15xyz.text = f'+{formatted_value15xyz}'
    else:    
        cell_col15xyz.text = '-'
    
    paragraph_col15xyz = cell_col15xyz.text_frame.paragraphs[0]
    paragraph_col15xyz.font.size = Pt(14) 
    paragraph_col15xyz.font.bold = True 

    if xyzval_col_4[i] > 0:
        paragraph_col15xyz.font.color.rgb = RGBColor(0, 176, 80) 
    elif xyzval_col_4[i] < 0:
        paragraph_col15xyz.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col15xyz.font.color.rgb = RGBColor(0, 0, 0)
    
    paragraph_col15xyz.font.name = 'Calibri' 
    paragraph_col15xyz.alignment = PP_ALIGN.CENTER

#col names 

# Actualizar encabezados de tabla con el mes y año de new_date_cell
tabla12xyz.cell(0, 1).text = f"Stock                 {current_month}-{current_year}"
tabla12xyz.cell(0, 3).text = f"TP                       {current_month}-{current_year}"

# Ajustar el formato del texto de los encabezados
for col in [1, 3]:
    paragraph12xyz = tabla12xyz.cell(0, col).text_frame.paragraphs[0]
    paragraph12xyz.font.size = Pt(12)
    paragraph12xyz.font.bold = True
    paragraph12xyz.font.color.rgb = RGBColor(84, 21, 98)  
    paragraph12xyz.font.name = 'Calibri'
    paragraph12xyz.alignment = PP_ALIGN.CENTER

    cell12xyz = tabla12xyz.cell(0, col)
    text_frame12xyz = cell12xyz.text_frame  
    text_frame12xyz.word_wrap = True  # Forzar el ajuste de línea
    text_frame12xyz.auto_size = None
    

#Table 2
tabla22xyz = diapositiva12xyz.shapes[6].table 

for i in range(len(xyzval_col_5)):

    #col
        formatted_value20xyz = f"{xyzval_col_5[i]:,}%".replace(",", " ")  
        
        cell_col20xyz = tabla22xyz.cell(i + 0, 1)  

        if xyzval_col_5[i] < 0:
            cell_col20xyz.text = formatted_value20xyz
        elif xyzval_col_5[i] > 0:
            cell_col20xyz.text = f'+{formatted_value20xyz}'
        else:    
            cell_col20xyz.text = '-'
        
        paragraph_col20xyz = cell_col20xyz.text_frame.paragraphs[0]
        paragraph_col20xyz.font.size = Pt(14) 
        paragraph_col20xyz.font.bold = True 

        if xyzval_col_5[i] > 0:
            paragraph_col20xyz.font.color.rgb = RGBColor(0, 176, 80) 
        elif xyzval_col_5[i] < 0:
            paragraph_col20xyz.font.color.rgb = RGBColor(192, 0, 0)
        else:
            paragraph_col20xyz.font.color.rgb = RGBColor(0, 0, 0)

        paragraph_col20xyz.font.name = 'Calibri' 
        paragraph_col20xyz.alignment = PP_ALIGN.CENTER

# Ajustar el ancho
position_tablaxyz = diapositiva12xyz.shapes[1]
position_tablaxyz.left = Inches(2.1)

tabla12xyz.columns[0].width = Inches(5.7)
tabla12xyz.columns[1].width = Inches(1.19)
tabla12xyz.columns[2].width = Inches(1.16)
tabla12xyz.columns[3].width = Inches(1.16)
tabla12xyz.columns[4].width = Inches(1.15)

In [39]:
#BarChart
diapositiva16xyz = prs.slides[17]

chart16xyz = diapositiva16xyz.shapes[0].chart

# Nuevos datos para el gráfico de barras
chart_data16xyz = CategoryChartData()
chart_data16xyz.categories = name_site
chart_data16xyz.add_series(f'{month[:3]}-{year % 100}', xyval_col_1)

chart16xyz.replace_data(chart_data16xyz)

plot16xyz = chart16xyz.plots[0]
plot16xyz.has_data_labels = False

#Table 1
tabla16xyz = diapositiva16xyz.shapes[1].table 

for i in range(len(xyzval_col_1)):
#col 1
    formatted_value16xyz = f"{xyzval_col_1[i]:,}".replace(",", " ")  
    
    cell_col16xyz = tabla16xyz.cell(i + 1, 1)  
    cell_col16xyz.text = formatted_value16xyz  
    
    paragraph_col16xyz = cell_col16xyz.text_frame.paragraphs[0]
    paragraph_col16xyz.font.size = Pt(14) 
    paragraph_col16xyz.font.bold = True 
    paragraph_col16xyz.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col16xyz.font.name = 'Calibri' 
    paragraph_col16xyz.alignment = PP_ALIGN.CENTER

#col 2
    formatted_value17xyz = f"{xyzval_col_6[i]:,}%".replace(",", " ")
    
    cell_col17xyz = tabla16xyz.cell(i + 1, 2)  

    if xyzval_col_6[i] < 0:
        cell_col17xyz.text = formatted_value17xyz
    elif xyzval_col_6[i] > 0:
        cell_col17xyz.text = f'+{formatted_value17xyz}'
    else:    
        cell_col17xyz.text = '-'
    
    paragraph_col17xyz = cell_col17xyz.text_frame.paragraphs[0]
    paragraph_col17xyz.font.size = Pt(14) 
    paragraph_col17xyz.font.bold = True

    if xyzval_col_6[i] > 0:
        paragraph_col17xyz.font.color.rgb = RGBColor(0, 176, 80) 
    elif xyzval_col_6[i] < 0:
        paragraph_col17xyz.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col17xyz.font.color.rgb = RGBColor(0, 0, 0)

    paragraph_col17xyz.font.name = 'Calibri'
    paragraph_col17xyz.alignment = PP_ALIGN.CENTER

#col 3
    formatted_value18xyz= f"{xyzval_col_3[i]:,}%".replace(",", " ")  
    
    cell_col18xyz = tabla16xyz.cell(i + 1, 3)  
    cell_col18xyz.text = formatted_value18xyz  
    
    paragraph_col18xyz = cell_col18xyz.text_frame.paragraphs[0]
    paragraph_col18xyz.font.size = Pt(14) 
    paragraph_col18xyz.font.bold = True 
    paragraph_col18xyz.font.color.rgb = RGBColor(0, 0, 0)
    paragraph_col18xyz.font.name = 'Calibri' 
    paragraph_col18xyz.alignment = PP_ALIGN.CENTER

#col 4
    formatted_value19xyz = f"{xyzval_col_7[i]:,}%".replace(",", " ")  
    
    cell_col19xyz = tabla16xyz.cell(i + 1, 4)  

    if xyzval_col_7[i] < 0:
        cell_col19xyz.text = formatted_value19xyz
    elif xyzval_col_7[i] > 0:
        cell_col19xyz.text = f'+{formatted_value19xyz}'
    else:    
        cell_col19xyz.text = '-'
    
    paragraph_col19xyz = cell_col19xyz.text_frame.paragraphs[0]
    paragraph_col19xyz.font.size = Pt(14) 
    paragraph_col19xyz.font.bold = True 

    if xyzval_col_7[i] > 0:
        paragraph_col19xyz.font.color.rgb = RGBColor(0, 176, 80) 
    elif xyzval_col_7[i] < 0:
        paragraph_col19xyz.font.color.rgb = RGBColor(192, 0, 0)
    else:
        paragraph_col19xyz.font.color.rgb = RGBColor(0, 0, 0)
    
    paragraph_col19xyz.font.name = 'Calibri' 
    paragraph_col19xyz.alignment = PP_ALIGN.CENTER

#col names 
# Actualizar encabezados de tabla con el mes y año de new_date_cell
tabla16xyz.cell(0, 1).text = f"Stock              {current_month}-{current_year}"
tabla16xyz.cell(0, 3).text = f"TP                     {current_month}-{current_year}"

# Ajustar el formato del texto de los encabezados
for col in [1, 3]:
    paragraph16xyz = tabla16xyz.cell(0, col).text_frame.paragraphs[0]
    paragraph16xyz.font.size = Pt(12)
    paragraph16xyz.font.bold = True
    paragraph16xyz.font.color.rgb = RGBColor(84, 21, 98)  
    paragraph16xyz.font.name = 'Calibri Light'
    paragraph16xyz.alignment = PP_ALIGN.CENTER

    cell16xyz = tabla16xyz.cell(0, col)
    text_frame16 = cell16xyz.text_frame  
    text_frame16.word_wrap = True
    text_frame16.auto_size = None
    
#Table 2
tabla55xyz = diapositiva16xyz.shapes[6].table 

for i in range(len(xyzval_col_5)):

    #col
        formatted_value20xyz = f"{xyzval_col_8[i]:,}%".replace(",", " ")  
        
        cell_col20xyz = tabla55xyz.cell(i + 0, 1)  

        if xyzval_col_8[i] < 0:
            cell_col20xyz.text = formatted_value20xyz
        elif xyzval_col_8[i] > 0:
            cell_col20xyz.text = f'+{formatted_value20xyz}'
        else:    
            cell_col20xyz.text = '-'
        
        paragraph_col20xyz = cell_col20xyz.text_frame.paragraphs[0]
        paragraph_col20xyz.font.size = Pt(14) 
        paragraph_col20xyz.font.bold = True 

        if xyzval_col_8[i] > 0:
            paragraph_col20xyz.font.color.rgb = RGBColor(0, 176, 80) 
        elif xyzval_col_8[i] < 0:
            paragraph_col20xyz.font.color.rgb = RGBColor(192, 0, 0)
        else:
            paragraph_col20xyz.font.color.rgb = RGBColor(0, 0, 0)

        paragraph_col20xyz.font.name = 'Calibri' 
        paragraph_col20xyz.alignment = PP_ALIGN.CENTER

# Ajustar el ancho
position_tablaxyz2 = diapositiva16xyz.shapes[1]
position_tablaxyz2.left = Inches(2.1)

tabla16xyz.columns[0].width = Inches(5.7)
tabla16xyz.columns[1].width = Inches(1.19)
tabla16xyz.columns[2].width = Inches(1.16)
tabla16xyz.columns[3].width = Inches(1.16)
tabla16xyz.columns[4].width = Inches(1.15)

<h3>Line Chart 2</h3>

In [40]:
ddwNew.iloc[1:, 1:] = ddwNew.iloc[1:, 1:].apply(pd.to_numeric, errors='coerce')

ddw_new_val_line_chart = ddwNew.iloc[:-2, -1].values.astype(int)

print(ddw_new_val_line_chart)

[14489  1490  1304  7502 15241  2939  1908   414  1064  2953]


In [41]:
# Access the specific slide with the chart (adjust the index as necessary)
diapositiva17 = prs.slides[18]

# Locate the chart in the slide
chart71 = None
for shape in diapositiva17.shapes:
    if shape.has_chart:
        chart71 = shape.chart
        break

# Excel date conversion function
def excel_date_to_datetime(excel_serial_date):
    base_date1 = datetime(1899, 12, 30)  # Excel's base date
    return base_date1 + timedelta(days=excel_serial_date)

# Extract category labels (dates)
plot71 = chart71.plots[0]
category_labels1 = [c.label for c in plot71.categories]
formatted_dates1 = [excel_date_to_datetime(int(date)).strftime('%Y-%m-%d') for date in category_labels1]
formatted_dates21 = [excel_date_to_datetime(int(date)).strftime('%b-%y') for date in category_labels1]

# Append new date to the category labels
new_date_form1 = datetime.strptime(new_date_cell, '%Y-%m-%d')
new_formatted1 = new_date_form1.strftime('%b-%y')
formatted_dates21.append(new_formatted1)

# Ensure only the last 12 months are kept
formatted_dates21 = formatted_dates21[-13:]

# Extract series names and values
x = []
x1 = []

for i in range(10):  # Loop over all 10 series
    series1 = chart71.series[i]
    x.append(series1.name)
    x1.append(list(series1.values))

# Initialize a dictionary to hold the company data
company_data1 = {}

for i in range(10):
    series1 = plot71.series[i]
    company_name1 = series1.name
    values1 = [int(v) if v is not None else 0 for v in series1.values]
    company_data1[company_name1] = values1

# Append the new data (12 months max)
for i, company in enumerate(company_data1.keys()):
    if i < len(ddw_new_val_line_chart):
        company_data1[company].append(ddw_new_val_line_chart[i])

    # Trim to the last 12 months for each series
    company_data1[company] = company_data1[company][-13:]

# Create a new CategoryChartData object
chart_data111 = CategoryChartData()

# Update categories to the last 12 months
chart_data111.categories = formatted_dates21

# Add updated series data (trimmed to 12 months)
for company, values in company_data1.items():
    chart_data111.add_series(company, values)

# Replace the chart data with the updated values
chart71.replace_data(chart_data111)

# Print to verify
print(company_data1)
print(category_labels1)
print(formatted_dates1)
print(formatted_dates21)



{'Argus': [13806, 14128, 14385, 14322, 14067, 14199, 14040, 14145, 14711, 14784, 15092, 14611, np.int64(14489)], 'AutoScout24': [1454, 1411, 1440, 1484, 1274, 1503, 1540, 1571, 1607, 1648, 1689, 1494, np.int64(1490)], 'Heycar': [797, 821, 848, 985, 1126, 1273, 1218, 1217, 1236, 1306, 1294, 1298, np.int64(1304)], 'La Centrale-Caradisiac': [7194, 7170, 7269, 7403, 7118, 7024, 7425, 7367, 7341, 7383, 7558, 7524, np.int64(7502)], 'Leboncoin': [14767, 15119, 15404, 15391, 15266, 15354, 15716, 15690, 15825, 15738, 15738, 15531, np.int64(15241)], 'OuestFrance': [2500, 2556, 2826, 2932, 2895, 2861, 2856, 2933, 2928, 2918, 2930, 2943, np.int64(2939)], 'ParuVendu': [1662, 1625, 1606, 1670, 1575, 1724, 1632, 1667, 1777, 1866, 1928, 1905, np.int64(1908)], 'Renault Occasions': [0, 0, 0, 0, 411, 414, 413, 413, 416, 414, 415, 418, np.int64(414)], 'Spoticar': [0, 0, 0, 0, 913, 1101, 1059, 1058, 1061, 1068, 1073, 1051, np.int64(1064)], 'Zoomcar': [2603, 2614, 2756, 2926, 2895, 2861, 2858, 2935, 2966, 2

<h3>Modifying table Dealer per website by size</h3>

In [42]:
# Nombre de la hoja
nombre_hoja = 'Pro size'

# Leer el archivo Excel y seleccionar la hoja por su nombre
ddws = pd.read_excel(xcl, sheet_name=nombre_hoja, header=1, index_col=0)

# Formatear números con separadores de miles
def format_number(value):
    if isinstance(value, (int, float)):
        return f"{value:,}".replace(",", " ")
    return value

ddws = ddws.applymap(format_number)

# Access the specific slide with the chart (adjust the index as necessary)
diapositiva19 = prs.slides[19]

# Locate the chart in the slide
chart_t = None
for shape in diapositiva19.shapes:
    if shape.has_table:
        chart_t = shape.table
        break

if chart_t:
    # Iterate through the table and replace its data
    row_count = len(chart_t.rows)
    col_count = len(chart_t.columns)

    # Prepare the data for replacement (excluding website names)
    data_columns = ['<10 Listings', '10-20 Listings', '21-50 Listings', '> 50 Listings']
    ddws = ddws.reset_index(drop=True)

    for i, row in ddws.iterrows():
        if i + 1 >= row_count:  # Ensure we do not exceed the table rows
            break

        for j, col_name in enumerate(data_columns):
            if j + 1 >= col_count:  # Ensure we do not exceed the table columns
                break
            cell = chart_t.cell(i + 1, j + 1)
            cell.text = str(row[col_name])
            
            # Align text to the center
            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
            
            # Set font to Calibri, size 18
            run = cell.text_frame.paragraphs[0].runs[0]
            run.font.name = 'Calibri'
            run.font.size = Pt(18)

    print("Data replaced successfully!")
else:
    print("No table found in slide 19.")

Data replaced successfully!


C:\Users\r.claros\AppData\Local\Temp\ipykernel_15896\2196503919.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ddws = ddws.applymap(format_number)


<h3>Shared dealers between websites </h3>

In [43]:
# Load Excel data into DataFrame, force float to int directly
sheet_name = 'Commun pros'
ddbw = pd.read_excel(xcl, sheet_name=sheet_name, header=1, index_col=1).reset_index()
if ddbw.columns[0] == 'Unnamed: 0':
    ddbw = ddbw.drop(columns=ddbw.columns[0])
ddbw.columns = [pd.to_datetime(col).strftime('%B %Y') if '00:00:00' in str(col) else col for col in ddbw.columns]
ddbw = ddbw.astype(str)

In [44]:
# Formatear números con separadores de miles
def format_number(value):
    if isinstance(value, (int, float)):
        return f"{value:,}".replace(",", " ")
    return value

ddbw = ddbw.applymap(format_number)

# Access the specific slide with the chart (adjust the index as necessary)
diapositiva19x = prs.slides[20]

# Locate the chart in the slide
chart_t = None
for shape in diapositiva19x.shapes:
    if shape.has_table:
        chart_t = shape.table
        break

if chart_t:
    # Iterate through the table and replace its data
    row_count = len(chart_t.rows)
    col_count = len(chart_t.columns)

    # Prepare the data for replacement (excluding website names)
    data_columns = ["Argus", "AutoScout24", "Heycar", "La Centrale", "Leboncoin", "OuestFrance", "ParuVendu",
                     "Renault Occasions", "Spoticar", "Zoomcar"]
    ddbw = ddbw.reset_index(drop=True)

    for i, row in ddbw.iterrows():
        if i + 1 >= row_count:  # Ensure we do not exceed the table rows
            break

        for j, col_name in enumerate(data_columns):
            if j + 1 >= col_count:  # Ensure we do not exceed the table columns
                break
            
            cell = chart_t.cell(i + 1, j + 1)
            cell.text = str(row[col_name])
            
            # Align text to the center
            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
            
            # Set font to Arial Narrow, size 14
            run = cell.text_frame.paragraphs[0].runs[0]
            run.font.name = 'Arial Narrow'
            run.font.size = Pt(14)
            
            # Apply bold formatting to diagonal cells
            if i == j:  # Diagonal condition
                run.font.bold = True

    print("Data replaced successfully!")
else:
    print("No table found in slide 9.")

# Guardar la presentación con los cambios
prs.save(f"ZZ{pd.Timestamp.now().strftime('%Y%m%d%H%M%S')}_auto_ppt.pptx")

# limpiar los archivos temporales
try:
    shutil.rmtree(temp_folder)
    os.remove(temp_pptx_path)
    print("Temporary files removed.")
except Exception as e:
    print(f"Error during cleanup: {e}")


C:\Users\r.claros\AppData\Local\Temp\ipykernel_15896\2730333204.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ddbw = ddbw.applymap(format_number)


Data replaced successfully!
Temporary files removed.
